In [1]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import re
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup

import requests

import string
from collections import Counter

In [2]:
r = requests.get('https://midasfurniture.com/ksa_ar/neira-sectional.html')
soup = BeautifulSoup(r.text, 'html.parser')
description = soup.find('div', {'itemprop': 'description'})
description

<div class="value" itemprop="description"><p><b>نظرة عامة : </b>قسمت هذه الاريكة ذات القماش الناعم بشكل واسلوب رائعان, ودعمت بوسائد متنوعه ومميزه لطبقة إضافية من الراحة اللون والملمس<br/><br/><b>الوصف : </b>طقم زاوية ثلاثة قطع <br/><br/><b>تفاصيل المنتج :</b><br/>♦ كنبة ثلاثة مقاعد يد يمين عدد 2 قطعة<br/>♦ اوتومان عدد 1 قطعة<br/>♦ وسادة مربعة للديكور عدد 4 قطعة<br/>♦ وسادة مستطيلة للديكور عدد 2 قطعة <br/><br/><b>حجم المنتج :</b><br/>♦ كنبة ثلاثة مقاعد يد يمين: العرض 203 * العمق 101 * الارتفاع 94 سم<br/>♦ اوتومان: العرض 99 * العمق 99 * الارتفاع 48 سم<br/><br/><b>الأقمشة : </b><br/>♦ بوليستر <br/><br/><b>اللون :</b><br/>♦ بيج</p>
<p><b>النوع :</b><br/>♦ انتقالي</p></div>

In [131]:
for elem in soup(text=re.compile(r'اللون')):
    print(elem.next_element.next_element)

<br/>
♦ بيج


In [112]:
def extract_element(name, soup):
    regex = re.compile(f'{name}')
    result = None
    for elem in soup(text=regex):
        try:
            result = elem.next_element.next_element.replace('♦', '').strip()
        except:
            result = elem.next_element.next_element.next_element.replace('♦', '').strip()
    return  result
free_colors = extract_element('اللون', soup)
style = extract_element('النوع', soup)

TypeError: 'NoneType' object is not callable

In [109]:

free_colors, style


('بيج', 'معاصر')

In [2]:
list_url = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/Midas_update_14_12_2021finalv1.xlsx')

In [4]:
toto.columns

Index(['sku number only', 'sku', 'store_view_code', 'attribute_set_code',
       'product_type', 'product_websites', 'name', 'estimated_delivery_enable',
       'estimated_delivery_text', 'url_key', 'description',
       'short_description', 'Link_url', 'categories1', 'categories2',
       'categories3', 'categories', 'free_colors', 'raw_materials',
       'ts_dimensions_width', 'ts_dimensions_height', 'ts_dimensions_length',
       'cost', 'price', 'special_price', 'visibility', 'tax_class_name',
       'manufacturer', 'news_from_date', 'news_to_date', 'base_image',
       'small_image', 'swatch_image', 'thumbnail_image', 'additional_images ',
       'product_online', 'qty', 'out_of_stock_qty', 'allow_backorders',
       'is_in_stock', 'style', 'supplier'],
      dtype='object')

In [14]:

button = driver.find_element_by_css_selector( '.sc-xf6b4m-3')
button.click()

In [16]:
len(list_url)

6156

In [17]:

def return_details(value, list_dic):
        val = ''
        for i in list_dic:
            if value in i:
                val = i.replace(value, '')
        return val
count = 0
name_excel = 'midas_update_en2.xlsx'
df = pd.read_excel('midas3_model.xlsx')
#ForLoop main
for i, product in enumerate(list_url[5810:]):
#for i, product in enumerate(list_to_products):
    time.sleep(1)
    try:
        print('*************************************')
        print('Count:', i)
        url = product.replace('ksa_ar', 'ksa_en')
        driver.get(url)
        time.sleep(1)
    except:
        continue
    try:
        name = driver.find_element_by_class_name('product-name').text
    
        link_url = str(product[0])


        sku = driver.find_element_by_class_name('sku').text[4:]
        sku = 'DAS-' + sku 
        url_key = (f'{sku}-{name}').replace(',', '-').replace('*', '-').replace('،', '-').replace('/', '-').replace('%', '').replace('"', '')
    
        combination = driver.find_element_by_class_name('std').text
    except:
        continue
    try:
        combination_product_size_list = combination.split('\n')
        combination_product_size_list1 = driver.find_element_by_class_name('std').text.split('\n\n')
        combination = combination_product_size_list[1].replace('♦', '')
    except:
        combination_product_size_list = combination.split('\n')
        combination_product_size_list1 = driver.find_element_by_class_name('std').text.split(':')
        combination = combination_product_size_list[0].replace('♦', '')
        
    #product_size = combination_product_size_list[4].replace('♦', '')
# DETAIL  *******************************************   
    
    detail = return_details('تفاصيل المنتج', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    if detail == '':
        for dt in combination_product_size_list1:


            if dt.find('Combination') == 0:
                print('coucou')
                detail = dt.replace('♦ ', '').replace('Combination', '').replace(':', '').replace('\n', '')
    print('Detail: ',detail)   
# COLOR  *******************************************   

#*******************************************************
    series = pd.Series([sku,
                         name,
                         url,
                        detail,
                        ],  index=df.columns)
    #df.append(df1)
    df = df.append(series, ignore_index=True)
    df.to_excel( f'{name_excel}', index = False)


*************************************
Count: 0
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 1
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 2
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 3
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 4
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 5
coucou
Detail:   Rectangle Decorative Pillow 1 PCS
*************************************
Count: 6
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 7
coucou
Detail:   Rectangle Decorative Pillow 1 PCS
*************************************
Count: 8
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 9
coucou
Detail:   Rectangle Decorative Pillow 1 PCS
**************************

coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 85
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 86
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 87
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 88
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 89
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 90
coucou
Detail:   Rectangle Decorative Pillow 1 PCS
*************************************
Count: 91
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 92
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 93
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 94
coucou
Detail:   Squar

*************************************
Count: 168
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 169
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 170
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 171
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 172
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 173
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 174
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 175
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 176
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 177
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
***************

*************************************
Count: 254
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 255
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 256
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 257
*************************************
Count: 258
*************************************
Count: 259
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 260
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 261
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 262
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*************************************
Count: 263
coucou
Detail:   Square Decorative Pillow 1 PCS
*************************************
Count: 264
coucou
Detail:   Rectangular Decorative Pillow 1 PCS
*********

In [18]:
df1 = pd.read_excel("/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_en.xlsx")
df2 = pd.read_excel("/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_en2.xlsx")

In [19]:
df = pd.concat([df1, df2], ignore_index=True)

In [20]:
df.to_excel('Midas_update_en.xlsx')

# Scrape all url with BS4

In [10]:
links_products = []

list_cat1 = []
list_cat2 = []
list_cat3 = []
for i, toto in enumerate(list_dic_cats):
    print('Count:', i)
    for i in range(1, toto['count']):
        url = (f'{toto["url"]}{i}.html')
#         url = f'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sectional-sofa/page/{i}.html'
        r = requests.get(url)
        html_soup = BeautifulSoup(r.text, 'html.parser')
#         print(f'{toto[0]}{i}.html')

#         products = driver.find_elements_by_class_name('product-image')

        links = html_soup.find_all('a', class_='product-image')
        #Forloop to get all url products
        for link in links:
            links_products.append(link.get('href'))
            list_cat1.append(toto['cat1'])
            list_cat2.append(toto['cat2'])
            list_cat3.append(toto['cat3'])

        print('cat1', toto['cat1'])
        print('cat2', toto['cat2'])
        print('cat3', toto['cat3'])
#             time.sleep(0.5)time.sleep(1)
        print(len(links_products))
       
#Save link in excel files
link_living_rooms = {
    'liste': links_products,
    'cat1': list_cat1,
    'cat2': list_cat2,
    'cat3': list_cat3,
}
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
liste_link_living_rooms = pd.DataFrame(link_living_rooms)
liste_link_living_rooms.to_excel(f'link_all_product_midas_cats_titi1-{dt_string}.xlsx')

Count: 0
cat1  أثاث منزلي
cat2 غرف المعيشة
cat3  كنب زاوية
27
cat1  أثاث منزلي
cat2 غرف المعيشة
cat3  كنب زاوية
54
cat1  أثاث منزلي
cat2 غرف المعيشة
cat3  كنب زاوية
81
cat1  أثاث منزلي
cat2 غرف المعيشة
cat3  كنب زاوية
85
Count: 1
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  أطقم كنب
112
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  أطقم كنب
139
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  أطقم كنب
166
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  أطقم كنب
193
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  أطقم كنب
220
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  أطقم كنب
224
Count: 2
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3 كنب ثلاث مقاعد و مقعدين
251
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3 كنب ثلاث مقاعد و مقعدين
278
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3 كنب ثلاث مقاعد و مقعدين
300
Count: 3
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  كنب سرير
314
Count: 4
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  كراسي
341
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  كراسي
368
cat1 أثاث منزلي
cat2 غرف المعيشة
cat3  كراسي
395
cat1 أثاث منزلي
cat2 غرف المعيشة
c

cat1 أثاث مكتبي
cat2 مكاتب و مجموعة مكتبية
cat3 
2678
Count: 62
cat1 أثاث مكتبي
cat2 كنبات المكتب
cat3 
2705
cat1 أثاث مكتبي
cat2 كنبات المكتب
cat3 
2715
Count: 63
cat1 أثاث مكتبي
cat2 خزائن
cat3 
2734
Count: 64
cat1 أثاث مكتبي
cat2  طاولات الاجتماعات و الوسط و الطاولات الجانبية
cat3 
2761
cat1 أثاث مكتبي
cat2  طاولات الاجتماعات و الوسط و الطاولات الجانبية
cat3 
2773
Count: 65
cat1 أثاث مكتبي
cat2 إكسسوارات المكاتب
cat3 
2780
Count: 66
cat1 إكسسوارات
cat2 أدوات المطبخ
cat3 أطقم العشاء
2780
Count: 67
cat1 إكسسوارات
cat2 أدوات المطبخ
cat3 صحون التقديم
2807
cat1 إكسسوارات
cat2 أدوات المطبخ
cat3 صحون التقديم
2834
cat1 إكسسوارات
cat2 أدوات المطبخ
cat3 صحون التقديم
2851
Count: 68
cat1 إكسسوارات
cat2 أدوات المطبخ
cat3 أوعية تقديم
2878
cat1 إكسسوارات
cat2 أدوات المطبخ
cat3 أوعية تقديم
2905
cat1 إكسسوارات
cat2 أدوات المطبخ
cat3 أوعية تقديم
2932
cat1 إكسسوارات
cat2 أدوات المطبخ
cat3 أوعية تقديم
2942
Count: 69
cat1 إكسسوارات
cat2 أدوات المطبخ
cat3 تخزين البهارات
2947
Count: 70
cat1 إكسسوارات
cat2

cat1 إكسسوارات
cat2 وسائد وأغطية
cat3 وسائد
5975
cat1 إكسسوارات
cat2 وسائد وأغطية
cat3 وسائد
6002
cat1 إكسسوارات
cat2 وسائد وأغطية
cat3 وسائد
6004
Count: 110
cat1 إكسسوارات
cat2 وسائد وأغطية
cat3 شال
6018
Count: 111
cat1 إكسسوارات
cat2 وسائد وأغطية
cat3 مخدة سرير
6018
Count: 112
cat1 إكسسوارات
cat2 دواسات الباب
cat3 
6021
Count: 113
cat1 إكسسوارات
cat2 خزنات
cat3 
6048
cat1 إكسسوارات
cat2 خزنات
cat3 
6050
Count: 114
cat1 إكسسوارات فاخرة
cat2 فاخرة أكواب
cat3 
6055
Count: 115
cat1 إكسسوارات فاخرة
cat2 أطقم فناجين مع أطباق
cat3 
6082
cat1 إكسسوارات فاخرة
cat2 أطقم فناجين مع أطباق
cat3 
6102
Count: 116
cat1 إكسسوارات فاخرة
cat2 فناجين وأطباق
cat3 
6126
Count: 117
cat1 إكسسوارات فاخرة
cat2 أوانى طعام
cat3 
6153
cat1 إكسسوارات فاخرة
cat2 أوانى طعام
cat3 
6162
Count: 118
cat1 إكسسوارات فاخرة
cat2 موزعات المشروبات
cat3 
6166
Count: 119
cat1 إكسسوارات فاخرة
cat2 أطقم أكواب المشروبات
cat3 
6178
Count: 120
cat1 إكسسوارات فاخرة
cat2 أوعية تقديم
cat3 
6205
cat1 إكسسوارات فاخرة
cat2 أوعية تقديم
cat

In [4]:
midas_url = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/djangoSripte/media/Midas/midas_update_products_url_25_09_2021v2.xlsx')

In [5]:
list_dic_cats = []
for index, row in midas_url.iterrows():
    list_dic_cats.append({
        'url': row['url'],
        'cat1': row['cat1'],
        'cat2': row['cat2'],
        'cat3': row['cat3'],
        
    })

In [6]:
len(list_dic_cats)

6913

In [12]:
url =  'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sectional-sofa.html'


In [19]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
cookies = {'session': '134-8225175-0355220'}
r = requests.get(url, headers=headers, cookies=cookies)
soup = BeautifulSoup(r.content, "html.parser")

not_available = soup.find_all('li', {'class': 'item last'})
len(not_available)
list_div = []
for tt in not_available:
    try:
        if tt.find('p', {'class': 'availability out-of-stock'}).find('span').text == 'غير متوفر':
            list_div.append(tt)
    except:
        pass
    
data = []
for pp in list_div:
    name = print(pp.find('h2', {'class': 'product-name'}).find('a')['title'])
    sku =  print(pp.find('div', {'class': 'sku'}).find('span', {'itemprop': 'productID'}).text.strip())
    price = print(pp.find('span', {'class': 'price'}).text.replace('SAR', '').replace(',', '').strip())
    
    data.append[
        {
            'name': name,
            'sku': sku,
            'price': price
        }
    ]
    

27

In [47]:
list_dic_cats1 = [
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sectional-sofa.html',  'cat1': ' أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' كنب زاوية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sofa-sets.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' أطقم كنب'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sofas-love-seats.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'كنب ثلاث مقاعد و مقعدين'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sofa-beds.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' كنب سرير'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/chairs.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' كراسي'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/bench-and-ottoman.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'بنش و أوتومان'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/center-side-tables.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'أطقم طاولات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/center-tables.html',  'cat1': 'أثاث منزلي', 'cat2': ' غرف المعيشة', 'cat3': ' طاولات وسط'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/end-tables.html',  'cat1': 'أثاث منزلي', 'cat2': ' غرف المعيشة', 'cat3': 'طاولات جانبية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/consoles.html',  'cat1': 'أثاث منزلي', 'cat2': ' غرف المعيشة', 'cat3': ' طاولات كونسول'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/nested-tables.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'طاولات متداخلة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/stands.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'ستاندات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/tv-units.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' مكتبات وطاولات تلفزيون'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/cabinets.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'خزائن'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/shoe-cabinets.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'خزائن الأحذية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/shelves.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'مكتبات ووحدات رفوف'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/chaise-lounge.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'شيزلونج'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/king-size-bedroom-sets.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'أطقم غرف النوم حجم ملكى'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/queen-size-bedroom-sets.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'أطقم غرف النوم حجم كوين'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/beds.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'سرير'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/night-stands.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': ' كومودينو'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/dresser.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'تسريحة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/chest-of-drawers.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'وحدة ادراج'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/wardrobes.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': ' خزانة ملابس'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/jewellery-chest.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'وحدة أدراج مجوهرات'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/dining-table-sets.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'أطقم غرف الطعام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/dining-tables.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'طاولة طعام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/dining-chairs.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'كراسي الطعام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/buffet.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'بوفيه وقاعدة بوفيه'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/high-dining-table-sets.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'أطقم طاولات طعام مرتفعة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/high-stool-chairs.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'كراسي مرتفعة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/counter-tables.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'طاولات متعددة الإستخدامات'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/recliners/rocker-recliner.html',  'cat1': 'أثاث منزلي', 'cat2': 'كراسي استرخاء', 'cat3': 'كرسي استرخاء هزاز'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/recliners/recliner.html',  'cat1': 'أثاث منزلي', 'cat2': ' كراسي استرخاء', 'cat3': 'كرسي استرخاء'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/recliners/sofa-recliner.html',  'cat1': 'أثاث منزلي', 'cat2': 'كراسي استرخاء', 'cat3': 'كنبة استرخاء'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/recliners/power-massage-recliner.html',  'cat1': 'أثاث منزلي', 'cat2': 'كراسي استرخاء', 'cat3': 'كراسى استرخاء كهربائى و مساج'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/bedroom-set.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'أطقم غرف النوم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/beds.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': ' سرير'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/night-stands.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'كومودينو'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/dressers.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'تسريحة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/chest-of-drawers.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'وحدة أدراج'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/wardrobes.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'خزانة ملابس'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/study-desk-chairs.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'مكتب دراسة و كرسي'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/bookcases-shelves.html',  'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'مكتبات و أرفف'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/home-office/desks.html',  'cat1': 'أثاث منزلي', 'cat2': 'مكاتب منزلية', 'cat3': 'مكاتب'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/home-office/chairs.html',  'cat1': 'أثاث منزلي', 'cat2': 'مكاتب منزلية', 'cat3': 'كراسي'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/super-king-size-200-200cm.html',  'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 200×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/super-king-size-193-203cm.html',  'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': ' فرشات مقاس 193×203 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/king-size-180-200cm.html',  'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 180×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/queen-size-150-200cm.html',  'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 150×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/queen-size-160x200cm.html',  'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': ' فرشات مقاس 160×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/full-size-135-190.html',  'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 135×190 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/twin-size-120-200cm.html',  'cat1': 'أثاث منزلي ', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 120×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/single-size-90-100x190cm.html',  'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 90 & 100 سم'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/gaming-chairs.html',  'cat1': 'أثاث منزلي', 'cat2': 'طاولات وكراسى الألعاب', 'cat3': ''} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/pet-furniture.html',  'cat1': 'أثاث منزلي', 'cat2': 'أثاث الحيوانات الأليفة', 'cat3': ''} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/outdoor-furniture.html',  'cat1': 'أثاث منزلي', 'cat2': 'أثاث خارجي', 'cat3': ''} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-chairs/high-back.html',  'cat1': 'أثاث مكتبي' , 'cat2': 'كراسي المكتب ', 'cat3': 'ظهر مرتفع' } , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-chairs/medium-back.html',  'cat1': 'أثاث مكتبي' , 'cat2': 'كراسي المكتب ', 'cat3':  'ظهر متوسط'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-chairs/low-back.html',  'cat1': 'أثاث مكتبي', 'cat2': 'كراسي المكتب', 'cat3': 'ظهر منخفض', } , 
    
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/visitor-chairs.html',  'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'كرسي زائر'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/desks-office-systems.html',  'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'مكاتب و مجموعة مكتبية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-sofas.html',  'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'كنبات المكتب'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/cabinets.html',  'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'خزائن'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/meeting-center-end-tables.html',  'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': ' طاولات الاجتماعات و الوسط و الطاولات الجانبية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-accessories.html',  'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'إكسسوارات المكاتب'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/dinner-sets.html.html',  'cat3': 'أطقم العشاء', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/dinner-and-serving-plates.html',  'cat3': 'صحون التقديم', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/bowls.html',  'cat3': 'أوعية تقديم', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/condiment-storage.html',  'cat3': 'تخزين البهارات', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/cups-and-saucers.html',  'cat3': 'فناجين وأطباق الشاى والقهوة', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/tea-and-sugar-pots.html',  'cat3': 'ابريق شاي وسكر', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/trays.html',  'cat3': 'الصواني', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/tiered-stand.html',  'cat3': 'حامل تقديم', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/baskets.html',  'cat3': 'سلة', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/jars.html',  'cat3': 'مرطبان', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/cutlery-sets.html',  'cat3': 'أدوات تناول الطعام', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/napkin-rings-and-holders.html',  'cat3': 'مسكة منديل', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/drinking-glasses-jugs.html',  'cat3': 'أواني زجاجية وأباريق', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 

{'url': 'https://midasfurniture.com/ksa_ar/accessories/table-linen/table-runners.html',  'cat3': 'مفرش طاولة', 'cat1': 'إكسسوارات', 'cat2': 'مفرش طاولة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/table-linen/place-mats.html',  'cat3': 'حصيرة تحت الأطباق', 'cat1': 'إكسسوارات', 'cat2': 'مفرش طاولة'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/serving-trolleys.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'عربات التقديم'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom.html',  'cat3': 'صحن صابون', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/soap-dispenser.html',  'cat3': 'موزع صابون', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/tumbler-and-tooth-brush-holders.html',  'cat3': 'حامل فرشاة الأسنان', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/toilet-paper-and-brush-holder.html',  'cat3': 'حامل ورق رول و فرشاة', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/towel-holder-and-racks.html',  'cat3': 'حامل مناشف', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/towels.html',  'cat3': 'منشفة', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/laundry-baskets.html',  'cat3': 'سلال غسيل', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/tissue-box.html',  'cat3': 'صندوق محارم', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/waste-bins.html',  'cat3': 'سلة مهملات', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/bathroom-mats.html',  'cat3': 'سجاد الحمام', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/candle-holders.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'شمعدان'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/vases-and-flowers.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'مزهريات و أحواض نبات'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/artificial-flowers-and-trees.html',  'cat3': 'الزهور والنباتات', 'cat1': 'إكسسوارات', 'cat2': 'ورود ونباتات'} ,
{'url': 'https://midasfurniture.com/ksa_ar/accessories/artificial-flowers-and-trees/trees.html',  'cat3': 'الأشجار', 'cat1': 'إكسسوارات', 'cat2': 'ورود ونباتات'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/lighting/table-lamps.html',  'cat1': 'إكسسوارات', 'cat2': 'الإضاءة', 'cat3': 'أباجورة طاولة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/lighting/hanging-and-ceiling-lamps.html',  'cat1': 'إكسسوارات', 'cat2': 'الإضاءة', 'cat3': 'ثريا سقف'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/lighting/floor-lamps.html',  'cat1': 'إكسسوارات', 'cat2': 'الإضاءة', 'cat3': 'أباجورة أرضية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/lighting/wall-lamp.html',  'cat1': 'إكسسوارات', 'cat2': 'الإضاءة', 'cat3': 'اباجورة حائط'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/photo-frames.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'براويز الصور'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/wall-arts-and-paintings.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'اللوحات وديكور الحائط'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/mirrors.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'المرايا'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/coat-hangers-screens.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'علاقات الملابس والفواصل'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/clocks.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'الساعات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/jewellery-organisers-box.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'صناديق للمجوهرات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/deco-figurines-and-objects.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'قطع فنية وديكورات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/ashtrays.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'منفضة سجائر'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/bedding-set.html',  'cat1': 'إكسسوارات', 'cat2': 'وسائد وأغطية', 'cat3': 'أطقم مفارش السرير'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions.html',  'cat1': 'إكسسوارات', 'cat2': 'وسائد وأغطية', 'cat3': 'وسائد'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/throws.html',  'cat1': 'إكسسوارات', 'cat2': 'وسائد وأغطية', 'cat3': 'شال'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/sleeping-pillows.html',  'cat1': 'إكسسوارات', 'cat2': 'وسائد وأغطية', 'cat3': 'مخدة سرير'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/doormats.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'دواسات الباب'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/safes.html',  'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'خزنات'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/mugs.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'فاخرة أكواب'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/cups-and-saucer-sets.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أطقم فناجين مع أطباق'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/cups-and-saucers.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'فناجين وأطباق'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/dinnerware.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أوانى طعام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/drinking-dispenser.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'موزعات المشروبات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/drinking-glasses-sets.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أطقم أكواب المشروبات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/serving-bowls.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أوعية تقديم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/condiment-storage.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أوعية تخزين البهارات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/napkin-rings-and-holders.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'حامل مناديل المائدة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/trays.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'صوانى'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/dinner-sets.html',  'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أطقم أوانى الطعام'} ,  
]


In [58]:
def get_data(url):
    """
    Fonction to scrape all urls from itch categories
    Return Data
    """
#     s = HTMLSession()
#     r = s.get(url)
#     r.html.render(timeout= 30, sleep=1)
    print('Url:', url)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    cookies = {'session': '134-8225175-0355220'}
    r = requests.get(url, headers=headers, cookies=cookies)
    soup = BeautifulSoup(r.content, "html.parser")
    
    not_available = soup.find_all('li', {'class': 'item last'})
    list_div = []
    for tt in not_available:
        try:
            if tt.find('p', {'class': 'availability out-of-stock'}).find('span').text == 'غير متوفر':
                list_div.append(tt)
        except:
            pass

    data = []
    for pp in list_div:
        name = pp.find('h2', {'class': 'product-name'}).find('a')['title']
        sku =  pp.find('div', {'class': 'sku'}).find('span', {'itemprop': 'productID'}).text.strip()
        price = pp.find('span', {'class': 'price'}).text.replace('SAR', '').replace(',', '').strip()

        data.append(
            {
                'name': name,
                'sku': sku,
                'price': price
            }
        )
    return soup, data


def getnextpage(soup):
    """
    Check if next url exist else send None objects
    Return URL or None
    """
    page = soup.find('a', {'class': 'next i-next'})
    # print('Page', page)
    
    try:
        # if next url exist 
        url2 = str(page['href'])
        return url2
        # print('', url2)
    except:
        print('No Next')
        pass
    return url2 

In [56]:
def scrap_url_product(url1):
    
    
    

    url = url1['url']
    data = []
    while True:
        soup, urls_list = get_data(url)
        
        for toto in urls_list:
            name = toto['name']
            sku = toto['sku']
            price = toto['price']
            # print(f'URL:', toto)
            data.append({
            'name': name,
            'sku': sku,
            'price': price,
            })
        try:
            url = getnextpage(soup)
#             print('Url dans le while', url)
        except:
            break
    # print(data)
    return data
    print( f'Scrape done .')

In [59]:
name_excel = 'midas_not_available_product.xlsx'
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/Categories/midas_updata_cats.xlsx')
        
for i, toto in enumerate(list_dic_cats1):
    print('Count: ', i)
    
    data = scrap_url_product(toto)
    df1 = pd.DataFrame(data)
    
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel(name_excel)

Count:  0
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sectional-sofa.html
Data [{'name': 'طقم زاوية لوبيليا', 'sku': '91285', 'price': '10095.00'}, {'name': 'كنبة زاوية ماير', 'sku': '2914', 'price': '5295.00'}, {'name': 'كنبة زاوية بريفادا', 'sku': '134246', 'price': '26995.00'}, {'name': 'كنبة زاوية بوفاريان', 'sku': '130167', 'price': '8995.00'}, {'name': 'كنبة زاوية سولوود', 'sku': '127120', 'price': '12795.00'}]
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sectional-sofa/page/2.html
Data [{'name': 'طقم زاوية لوبيليا', 'sku': '91285', 'price': '10095.00'}, {'name': 'كنبة زاوية ماير', 'sku': '2914', 'price': '5295.00'}, {'name': 'كنبة زاوية بريفادا', 'sku': '134246', 'price': '26995.00'}, {'name': 'كنبة زاوية بوفاريان', 'sku': '130167', 'price': '8995.00'}, {'name': 'كنبة زاوية سولوود', 'sku': '127120', 'price': '12795.00'}, {'name': 'كنبة زاوية دوفيمونت', 'sku': '126866', 'price': '5500.00'}, {'name': 'طقم كنبة زاوية بوليا', 'sku': '1160

Data [{'name': 'كرسى إستقبال سيشن', 'sku': '128234', 'price': '2365.00'}, {'name': 'كرسى إستقبال سولوود', 'sku': '127121', 'price': '4700.00'}, {'name': 'كرسى إستقبال تريكلور', 'sku': '123737', 'price': '3495.00'}, {'name': 'كرسى ديانتا', 'sku': '123130', 'price': '2315.00'}, {'name': 'كرسى إستقبال اندسترا', 'sku': '123123', 'price': '3185.00'}, {'name': 'كرسى إستقبال جرونالي', 'sku': '121674', 'price': '2380.00'}, {'name': 'كرسى إستقبال كيندرا', 'sku': '120895', 'price': '3850.00'}, {'name': 'كرسى إستقبال والش', 'sku': '120555', 'price': '1975.00'}, {'name': 'كرسى إستقبال ميلورا', 'sku': '119056', 'price': '2850.00'}, {'name': 'كرسى إستقبال دي بورو', 'sku': '117226', 'price': '2235.00'}, {'name': 'كرسى إستقبال راكو', 'sku': '115727', 'price': '2295.00'}, {'name': 'كرسى إستقبال سونيك', 'sku': '115701', 'price': '2965.00'}]
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/chairs/page/6.html
Data [{'name': 'كرسى إستقبال سيشن', 'sku': '128234', 'price': '2365.00'}, {'nam

Data [{'name': 'طقم طاولات قهوة 3 قطع ناشبرين', 'sku': '127940', 'price': '2885.00'}, {'name': 'طقم طاولات وسط 3 قطع امبروش', 'sku': '127936', 'price': '2150.00'}, {'name': 'طقم طاولات وسط 3 قطع روتكينز', 'sku': '123144', 'price': '1250.00'}, {'name': 'كلينكو', 'sku': '113117', 'price': '575.00'}, {'name': 'اوجاوا', 'sku': '112670', 'price': '1950.00'}]
No Next
Count:  7
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/center-tables.html
Data [{'name': 'مونزا سيلفر', 'sku': '4722', 'price': '3595.00'}]
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/center-tables/page/2.html
Data [{'name': 'مونزا سيلفر', 'sku': '4722', 'price': '3595.00'}]
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/center-tables/page/3.html
Data [{'name': 'مونزا سيلفر', 'sku': '4722', 'price': '3595.00'}, {'name': 'طاولة وسط كليمنت', 'sku': '122766', 'price': '4790.00'}, {'name': 'طاولة وسط كليريك', 'sku': '119145', 'price': '1850.00'}, {'name': 'طاولة وسط تي

Data [{'name': 'طاولة جانبية كوينا', 'sku': '127992', 'price': '795.00'}, {'name': 'طاولة جانبية كوينا', 'sku': '127991', 'price': '795.00'}, {'name': 'طاولة جانبية جيرميرا', 'sku': '127989', 'price': '795.00'}, {'name': 'طاولة جانبية كلارتو', 'sku': '127749', 'price': '475.00'}, {'name': 'طاولة جانبية لوكس', 'sku': '126835', 'price': '495.00'}, {'name': 'طاولة جانبية كليمنت', 'sku': '122767', 'price': '1950.00'}, {'name': 'طاولة جانبيه كليريك', 'sku': '119144', 'price': '780.00'}, {'name': 'طاولة جانبية فاليرى', 'sku': '119115', 'price': '2950.00'}, {'name': 'طاولة جانبيه البير', 'sku': '118994', 'price': '560.00'}, {'name': 'طاولة جانبية تريبو', 'sku': '118791', 'price': '375.00'}, {'name': 'طاولة جانبية كينيدي', 'sku': '118789', 'price': '615.00'}, {'name': 'طاولة جانبية موريلس', 'sku': '101519', 'price': '995.00'}, {'name': 'طاولة جانبية رنولدا ', 'sku': '127006', 'price': '985.00'}]
No Next
Count:  9
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/consoles.html


Data []
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/cabinets/page/2.html?gclid=EAIaIQobChMI-ZPx3d3F8gIVxhLTCh0_fgAKEAEYASAAEgIHnPD_BwE
Data [{'name': 'رف حائط-هارفى بارك', 'sku': '117960', 'price': '415.00'}, {'name': 'لوندون بوث خزانة عرض', 'sku': '115568', 'price': '625.00'}]
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/cabinets/page/3.html?gclid=EAIaIQobChMI-ZPx3d3F8gIVxhLTCh0_fgAKEAEYASAAEgIHnPD_BwE
Data [{'name': 'رف حائط-هارفى بارك', 'sku': '117960', 'price': '415.00'}, {'name': 'لوندون بوث خزانة عرض', 'sku': '115568', 'price': '625.00'}, {'name': 'ديسك بينك', 'sku': '110692', 'price': '2240.00'}]
No Next
Count:  14
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/shoe-cabinets.html
Data [{'name': 'خزانة أحذية ديسكريت', 'sku': '119032', 'price': '415.00'}]
No Next
Count:  15
Url: https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/shelves.html
Data []
Url: https://midasfurniture.com/ksa_ar/home-furniture/liv

Data [{'name': 'كرسي طعام فولتون', 'sku': '18541', 'price': '995.00'}]
Url: https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/dining-chairs/page/3.html
Data [{'name': 'كرسي طعام فولتون', 'sku': '18541', 'price': '995.00'}]
Url: https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/dining-chairs/page/4.html
Data [{'name': 'كرسي طعام فولتون', 'sku': '18541', 'price': '995.00'}]
No Next
Count:  28
Url: https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/buffet.html
Data [{'name': 'بريزدينت فارجاس', 'sku': '99190', 'price': '2500.00'}, {'name': 'بوفيه فازاني', 'sku': '128478', 'price': '1095.00'}]
Url: https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/buffet/page/2.html
Data [{'name': 'بريزدينت فارجاس', 'sku': '99190', 'price': '2500.00'}, {'name': 'بوفيه فازاني', 'sku': '128478', 'price': '1095.00'}, {'name': 'قاعدة بوفيه كريستا', 'sku': '119103', 'price': '5895.00'}, {'name': 'بوفيه سترومن', 'sku': '116856', 'price': '840.00'}, {'name': 'بوفيه أل

Data [{'name': 'كرسى زائر ستوك', 'sku': '67329', 'price': '745.00'}, {'name': 'كرسى زائر ستوك', 'sku': '67301', 'price': '745.00'}, {'name': 'كرسى زائر ديبوني', 'sku': '54993', 'price': '1207.50'}, {'name': 'كرسي زائر جراند', 'sku': '49773', 'price': '495.00'}, {'name': 'كرسى زائر مكتبى ساجا', 'sku': '132516', 'price': '1240.00'}]
Url: https://midasfurniture.com/ksa_ar/office-furniture/visitor-chairs/page/2.html
Data [{'name': 'كرسى زائر ستوك', 'sku': '67329', 'price': '745.00'}, {'name': 'كرسى زائر ستوك', 'sku': '67301', 'price': '745.00'}, {'name': 'كرسى زائر ديبوني', 'sku': '54993', 'price': '1207.50'}, {'name': 'كرسي زائر جراند', 'sku': '49773', 'price': '495.00'}, {'name': 'كرسى زائر مكتبى ساجا', 'sku': '132516', 'price': '1240.00'}, {'name': 'كرسى زائر مكتبى رون', 'sku': '132314', 'price': '1135.00'}, {'name': 'كرسى زائر مكتبى دينامو', 'sku': '128758', 'price': '1935.00'}]
No Next
Count:  61
Url: https://midasfurniture.com/ksa_ar/office-furniture/desks-office-systems.html
Data [{

Data []
Url: https://midasfurniture.com/ksa_ar/accessories/table-linen/place-mats/page/2.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/table-linen/place-mats/page/3.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/table-linen/place-mats/page/4.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/table-linen/place-mats/page/5.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/table-linen/place-mats/page/6.html
Data []
No Next
Count:  81
Url: https://midasfurniture.com/ksa_ar/accessories/serving-trolleys.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/serving-trolleys/page/2.html
Data []
No Next
Count:  82
Url: https://midasfurniture.com/ksa_ar/accessories/bathroom.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/bathroom/page/2.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/bathroom/page/3.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/bathroom/page/4.html
Data []
No

Data []
Url: https://midasfurniture.com/ksa_ar/accessories/wall-arts-and-paintings/page/5.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/wall-arts-and-paintings/page/6.html
Data [{'name': 'ديكور حائط سابينا', 'sku': '127048', 'price': '765.00'}]
Url: https://midasfurniture.com/ksa_ar/accessories/wall-arts-and-paintings/page/7.html
Data [{'name': 'ديكور حائط سابينا', 'sku': '127048', 'price': '765.00'}]
Url: https://midasfurniture.com/ksa_ar/accessories/wall-arts-and-paintings/page/8.html
Data [{'name': 'ديكور حائط سابينا', 'sku': '127048', 'price': '765.00'}]
Url: https://midasfurniture.com/ksa_ar/accessories/wall-arts-and-paintings/page/9.html
Data [{'name': 'ديكور حائط سابينا', 'sku': '127048', 'price': '765.00'}, {'name': 'ديكور حائط جاسكور', 'sku': '122694', 'price': '385.00'}, {'name': 'ديكور حائط ايداج', 'sku': '122687', 'price': '435.00'}, {'name': 'ديكور حائط سينورا', 'sku': '122681', 'price': '875.00'}]
Url: https://midasfurniture.com/ksa_ar/accessories/wall-a

Data [{'name': 'كلاند', 'sku': '18314', 'price': '27.00'}]
No Next
Count:  108
Url: https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/bedding-set.html
Data []
No Next
Count:  109
Url: https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions/page/2.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions/page/3.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions/page/4.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions/page/5.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions/page/6.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions/page/7.html
Data []
Url: https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions/page/8.html
Data []
Url: https://midasfurniture.com/ksa_ar/

In [60]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_not_available_product.xlsx')

In [62]:
toto = df.loc[df['price']<= 300]

In [67]:
toto['sku']  = toto['sku'].astype(str)

<ipython-input-67-128b34f63b19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toto['sku']  = toto['sku'].astype(str)


In [68]:
toto['sku']  = 'DAS-' + toto['sku'] 

<ipython-input-68-50a4981fc860>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toto['sku']  = 'DAS-' + toto['sku']


In [69]:
toto.to_excel('Midas_update_product_not_avail.xlsx')

In [ ]:
links_products = []

list_cat1 = []
list_cat2 = []
list_cat3 = []
for i, toto in enumerate(list_dic_cats):
    print('Count:', i)
    for i in range(1, toto['count']):
        url = (f'{toto["url"]}{i}.html')
#         url = f'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sectional-sofa/page/{i}.html'
        r = requests.get(url)
        html_soup = BeautifulSoup(r.text, 'html.parser')
#         print(f'{toto[0]}{i}.html')

#         products = driver.find_elements_by_class_name('product-image')

        links = html_soup.find_all('a', class_='product-image')
        #Forloop to get all url products
        for link in links:
            links_products.append(link.get('href'))
            list_cat1.append(toto['cat1'])
            list_cat2.append(toto['cat2'])
            list_cat3.append(toto['cat3'])

        print('cat1', toto['cat1'])
        print('cat2', toto['cat2'])
        print('cat3', toto['cat3'])
#             time.sleep(0.5)time.sleep(1)
        print(len(links_products))
       
#Save link in excel files
link_living_rooms = {
    'liste': links_products,
    'cat1': list_cat1,
    'cat2': list_cat2,
    'cat3': list_cat3,
}
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
liste_link_living_rooms = pd.DataFrame(link_living_rooms)
liste_link_living_rooms.to_excel(f'link_all_product_midas_cats_titi1-{dt_string}.xlsx')

In [60]:
len(articles)

92

In [ ]:
    
        driver.get(f'{toto[0]}{i}.html')
        print(f'{toto[0]}{i}.html')

        products = driver.find_elements_by_class_name('product-image')

In [52]:

len(links)

27

https://midasfurniture.com/ksa_ar/accrington-sectional-sofa.html
https://midasfurniture.com/ksa_ar/lobelia.html
https://midasfurniture.com/ksa_ar/tambo.html
https://midasfurniture.com/ksa_ar/tracling.html
https://midasfurniture.com/ksa_ar/maier-sectional-sofa.html
https://midasfurniture.com/ksa_ar/luxora.html
https://midasfurniture.com/ksa_ar/enola-sectional-sofa-with-chaise.html
https://midasfurniture.com/ksa_ar/kumasi-sectional-sofa.html
https://midasfurniture.com/ksa_ar/eltmann.html
https://midasfurniture.com/ksa_ar/jayceon-sectional-sofa-with-chaise.html
https://midasfurniture.com/ksa_ar/dovan.html
https://midasfurniture.com/ksa_ar/savesto.html
https://midasfurniture.com/ksa_ar/comfy-sectional-recliner-sofa.html
https://midasfurniture.com/ksa_ar/ballinasloe-sectional-sofa.html
https://midasfurniture.com/ksa_ar/altari-sectional-sofa.html
https://midasfurniture.com/ksa_ar/coulee-point-sectional-sofa.html
https://midasfurniture.com/ksa_ar/santasia-sectional-sofa.html
https://midasfurn

In [257]:
pp = pd.read_excel('/home/wafistos/Downloads/ميداس 11-10.xlsx')

In [264]:
pp.columns

Index(['sku', 'store_view_code', 'attribute_set_code', 'product_websites',
       'Unnamed: 4', 'name', 'estimated_delivery_enable',
       'estimated_delivery_text', 'url_key', 'description',
       'short_description', 'Link_url', 'categories1', 'categories2',
       'categories3', 'categories', 'free_colors', 'raw_materials',
       'ts_dimensions_width', 'ts_dimensions_height', 'ts_dimensions_length',
       'cost', 'price', 'special_price', 'visibility', 'tax_class_name',
       'manufacturer', 'news_from_date', 'news_to_date', 'base_image',
       'small_image', 'swatch_image', 'thumbnail_image', 'additional_images',
       'product_online', 'qty', 'out_of_stock_qty', 'allow_backorders',
       'is_in_stock', 'style', 'supplier'],
      dtype='object')

In [260]:
df = pd.read_csv('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/test_midas_clean_name.csv')

In [262]:
df.columns

Index(['Unnamed: 0', 'sku', 'name', 'url_key', 'short_description',
       'description', 'free_colors', 'raw_materials', 'ts_dimensions_width',
       'ts_dimensions_height', 'ts_dimensions_length', 'Weight', 'base_image',
       'small_image', 'swatch_image', 'thumbnail_image', 'additional_images ',
       'categories1', 'categories2', 'categories3', 'price', 'special_price',
       'visibility', 'tax_class_name', 'Link_url', 'qty', 'out_of_stock_qty',
       'is_in_stock', 'product_online', 'allow_backorders', 'manufacturer',
       'news_from_date', 'news_to_date', 'product_websites',
       'attribute_set_code', 'product_type', 'store_view_code', 'supplier',
       'style', 'First', 'Last', 'estimated_delivery_text'],
      dtype='object')

In [268]:
pp['categories'] = ''

In [276]:
list_cat = []
for index, row in df.iterrows():
    list_cat.append( {
        'sku': row['sku'],
        'cat1': row['categories1'],
        'cat2': row['categories2'],
        'cat3': row['categories3'],
    }, )

In [280]:
for tt in  list_cat:
    pp.loc[pp['sku'] == tt['sku'], 'categories1'] = tt['cat1']

In [282]:
pp.loc[pp['categories1']=='']

,sku,store_view_code,attribute_set_code,product_websites,Unnamed: 4,name,estimated_delivery_enable,estimated_delivery_text,url_key,description,...,swatch_image,thumbnail_image,additional_images,product_online,qty,out_of_stock_qty,allow_backorders,is_in_stock,style,supplier
20,DAS-122917,NaN,Default,base,NaN,طقم غرفة نوم بلمباجو سرير ملكى حجم 193X203 سم-,Static Text,هذا المنتج سيكون متوفر للتوصيل بعد 54 ايام,طقم غرفة نوم بلمباجو سرير ملكى حجم 193X203 سم-...,سرير 1 قطعة كمودينو 2 قطعة تسريحة 1 قطعة ...,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,2,0,-5,0,0,معاصر,DAS


In [309]:
pp.to_excel('Midas_update.xlsx')

In [308]:
pp['product_type'] = 'simple'

In [292]:
df1= pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_09_10_2021v2.xlsx')
df2= pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_09_10_2021.xlsx')

In [293]:
df = pd.concat([df1, df2], ignore_index=True)

In [294]:
df.shape

(6116, 38)

In [297]:
df.loc[df['sku']]

,sku,name,url_key,short_description,description,free_colors,raw_materials,ts_dimensions_width,ts_dimensions_height,ts_dimensions_length,...,allow_backorders,manufacturer,news_from_date,news_to_date,product_websites,attribute_set_code,product_type,store_view_code,supplier,style


In [298]:
pp.shape

(63, 41)

In [122]:
# base_url = 'https://midasfurniture.com/ksa_ar/home-furniture/'
# links_seen = set()
# def visit(url, links_seen):
#     html = requests.get(url).text
#     html_soup = BeautifulSoup(html, 'html.parser')
#     links_seen.add(url)
#     for link in html_soup.find_all("a"):
#         link_url = link.get('href')
#         if link_url is None:
#             continue
#         full_url = urljoin(url, link_url)
#         if full_url in links_seen:
#             continue
#         print('Found a new page:', full_url)
#         # Normally, we'd store the results here too
#         visit(full_url, links_seen)
        
# visit(base_url, links_seen)

Found a new page: https://midasfurniture.com/ar/
Found a new page: https://midasfurniture.com/ksa_ar/
Found a new page: https://midasfurniture.com/qtr_ar/
Found a new page: https://midasfurniture.com/jo_ar/


KeyboardInterrupt: 

# Firefox 

In [2]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)


Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36


In [87]:
driver.get('https://midasfurniture.com/ksa_ar/kodiak-sofa-set.html')

In [88]:
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, 'html.parser')
def urlimage(text):
    '''finds image url'''
    imageurl = []
    imageurl = re.findall(r'https?:\/\/.*\.(?:png|jpg)', text)
    return imageurl


In [89]:
images = soup.find_all('img', {'class': 'fotorama__img'})
len(images)

1

In [39]:
url_toto = 'https://midasfurniture.com/media/catalog/product/cache/7bd35ba35992fa136f6e5c8a3779eb8b/1/7/17-116196_b.jpg'

In [ ]:
ce18d1d07b138f93805957ee70f15be2

In [ ]:
images = soup.find_all('img', {'class': 'fotorama__img'})
len('')

In [118]:
driver.get('https://midasfurniture.com/ksa_ar/binova-office-chair-high-back.html')
time.sleep(5)
def extract_url_images(driver):
    r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    soup = BeautifulSoup(r, 'html.parser')
    images_base = soup.find('div', {'class': 'zoomWindow'})
    images_base = urlimage(str(images_base))[0]
    images_base = images_base.replace('https://midasfurniture.com/media/catalog/product/cache/', '')
    code_produit = images_base.split('/')[0]
    print('Code: ', code_produit)
    list_images = []
    # titit = url_toto.replace('https://midasfurniture.com/media/catalog/product/cache/', '')
    # titit = '/'.join(titit.split('/')[1: ])
    # titit = url_toto.replace('https://midasfurniture.com/media/catalog/product/cache/', '')
    # titit = '/'.join(titit.split('/')[1: ])
    images = soup.find_all('img', {'class': 'fotorama__img'})
    len(images)
    if len(images)>=8:
        images = images[:6]
    for img in images:
        url_toto1 = img['src']
        titit = url_toto1.replace('https://midasfurniture.com/media/catalog/product/cache/', '')
        titit = '/'.join(titit.split('/')[1: ])

        list_images.append('https://midasfurniture.com/media/catalog/product/cache/' + code_produit +'/'+ titit)
    return list_images
turl = extract_url_images(driver)
turl = list(dict.fromkeys(turl))
turl

Code:  ce18d1d07b138f93805957ee70f15be2


['https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/2/3/23-128743_a.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/2/3/23-128743_c.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/2/3/23-128743_b.jpg']

In [116]:
turl = extract_url_images(driver)
turl = list(dict.fromkeys(turl))
turl

Code:  ce18d1d07b138f93805957ee70f15be2


['https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-12718_a.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-12718_o.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-12718_b.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-12718_c.jpg']

In [38]:
for img in images:
    print(img['src'])

https://midasfurniture.com/media/catalog/product/cache/7bd35ba35992fa136f6e5c8a3779eb8b/1/7/17-116196_b.jpg
https://midasfurniture.com/media/catalog/product/cache/7bd35ba35992fa136f6e5c8a3779eb8b/1/7/17-116196_c.jpg
https://midasfurniture.com/media/catalog/product/cache/7bd35ba35992fa136f6e5c8a3779eb8b/1/7/17-116196_a.jpg
https://midasfurniture.com/media/catalog/product/cache/aaa7f864ab33e25e33f76ae89b6d0674/1/7/17-116196_b.jpg
https://midasfurniture.com/media/catalog/product/cache/aaa7f864ab33e25e33f76ae89b6d0674/1/7/17-116196_a.jpg
https://midasfurniture.com/media/catalog/product/cache/aaa7f864ab33e25e33f76ae89b6d0674/1/7/17-116196_c.jpg


In [10]:
lastElem = soup.find('b', text=re.compile('مواد التصنيع'))
lastElem

<b>مواد التصنيع :</b>

In [40]:
def urlimage(text):
    '''finds image url'''
    imageurl = []
    imageurl = re.findall(r'https?:\/\/.*\.(?:png|jpg)', text)
    return imageurl

In [42]:
images = driver.find_elements_by_xpath('//div[@class="fotorama__nav__frame fotorama__nav__frame--thumb"]')
list_images = []
if len(images) == 0:
    list_images.append(urlimage(driver.find_element_by_xpath('//div[@class="zoomWindowContainer"]').get_attribute('innerHTML')))
    
else:
    for img in images:
        img.click()
        time.sleep(1)
        list_images.append(urlimage(driver.find_element_by_xpath('//div[@class="zoomWindowContainer"]').get_attribute('innerHTML')))


['https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/2/1/21-gw-8ct-120-oak_3.jpg']


In [20]:
firstElem = soup.find('b', text=re.compile('حجم المنتج'))
firstElem

In [27]:
firstElem = soup.find('', text=re.compile('حجم المنتج'))
if not firstElem:
    firstElem = soup.find('', text=re.compile('الحجم'))
lastElem = soup.find('', text=re.compile('مواد التصنيع'))
p_tags = []
products_size = []
next = firstElem.next_element
while next != lastElem:
    
    if '♦' in next:
        print('next: ', next)
        p_tags.append(next)
    next = next.next_element


next:  ♦ مكتب * عرض 152 * عمق 61 * إرتفاع 77 سم


In [16]:
p_tags

['حجم المنتج :',
 <br/>,
 '♦ حصيرة تحت الأطباق * عرض 38 * عمق 38 * ارتفاع سم',
 <br/>,
 <br/>]

In [72]:
images = driver.find_elements_by_xpath('//div[@class="fotorama__nav-wrap fotorama__nav-wrap--horizontal"]//img')
print('Len: ',len(images))
time.sleep(2)
if len(images) >= 5:
    images = images[:5]

    list_images1 = []
    for img in images:
        try:
            mimi = extract_images()
        except:
            continue
        aa = img.get_attribute('src').replace('https://midasfurniture.com/media/catalog/product/cache/', '').split('/')
        aa[0] = mimi
        list_images1.append('/'.join(aa))
    list_images = ['https://midasfurniture.com/media/catalog/product/cache/' + ig for ig in list_images1]
elif len(images) == 0:
    time.sleep(0.5)
    list_images = []
    print('ppap')
    imagtoto = driver.find_element_by_xpath('//img[@class="fotorama__img"]')
    list_images.append(imagtoto.get_attribute('src'))
else:
    list_images1 = []
    for img in images:
        mimi = extract_images()
        aa = img.get_attribute('src').replace('https://midasfurniture.com/media/catalog/product/cache/', '').split('/')
        aa[0] = mimi
        list_images1.append('/'.join(aa))
    list_images = ['https://midasfurniture.com/media/catalog/product/cache/' + ig for ig in list_images1]
list_images

Len:  6


['https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/128148_d.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/128148_c.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/128148_f.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/128148_e.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/128148_a.jpg']

In [68]:
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, 'html.parser')

In [53]:
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, 'html.parser')

def extract_elem(name, soup):
    try:
        return str(soup.find('b', text=re.compile(name)).next_element.next_element.next_element).replace('♦', '').strip()
    except:
        return ''
raw_material = extract_elem('مواد التصنيع :', soup)
free_color = extract_elem('اللون :', soup)
style = extract_elem('النوع :', soup)
free_color, style

('رمادى', 'إنتقالى')

In [26]:
raw_material = extract_elem('مواد التصنيع :', soup)
free_color = extract_elem('اللون :', soup)
style = extract_elem('النوع :', soup)

raw_material, free_color, style

('خشب مصنع', 'أبيض', 'معاصر')

In [69]:
toto = soup.find('b', text=re.compile('وصف المنتج'))
toto

<b>وصف المنتج : </b>

In [64]:
sentence = [  'الوصف :', 'وصف المنتج : ', 'وصف المنتج: : ']
toto = ''
for sen in sentence:
    try:
        try:
            toto = soup.find('strong', text=re.compile(sen)).next_element.next_element.strip()
            break
        except:
            toto = soup.find('b', text=re.compile(sen)).next_element.next_element.strip()
            break
    except:
        continue
toto

''

In [58]:
cookie = {'name': 'GPS',
  'value': '2d69842073cd565b',
  'path': '/',
  'domain': '.nr-data.net',
  'secure': True,
  'httpOnly': True,
  'expiry': 1655368602,
  'sameSite': 'None'}


In [3]:
driver.get('https://midasfurniture.com/ksa_ar/clifford-console.html')

In [191]:
driver.get('https://midasfurniture.com/ksa_ar/jayceon-sectional-sofa-with-chaise.html')

In [6]:
driver.find_element_by_xpath('/html/body/div[1]/div/section/div/div[2]/div/div/div[2]/div[1]/div[2]/div[2]/a/div/span/div').click()

In [ ]:

def return_details(value, list_dic):
        val = ''
        for i in list_dic:
            if value in i:
                val = i.replace(value, '')
        return val
count = 0
name_excel = 'midas_update_14-12-0vtoto1.xlsx'
df = pd.read_excel('midas1_model.xlsx')
#ForLoop main
for i, product in enumerate(list_to_products[:1000]):
#for i, product in enumerate(list_to_products):
    time.sleep(1)
    try:
        print('*************************************')
        print('Count:', i)
        driver.get(product[0])
        time.sleep(1)
    except:
        continue
    try:
        name = driver.find_element_by_class_name('product-name').text
    
        link_url = str(product[0])

        in_stock = driver.find_element_by_class_name('availability').text

        sku = driver.find_element_by_class_name('sku').text[4:]
        sku = 'DAS-' + sku 
        url_key = (f'{sku}-{name}').replace(',', '-').replace('*', '-').replace('،', '-').replace('/', '-').replace('%', '').replace('"', '')
    
        price = driver.find_elements_by_class_name('price')[0].get_attribute('innerHTML').replace('SAR', '').replace('\n', '').replace(' ', '')

        special_price = driver.find_elements_by_class_name('price')[1].get_attribute('innerHTML').replace('SAR', '').replace('\n', '').replace(' ', '')

        combination = driver.find_element_by_class_name('std').text
    except:
        continue
    try:
        combination_product_size_list = combination.split('\n')
        combination_product_size_list1 = driver.find_element_by_class_name('std').text.split('\n\n')
        combination = combination_product_size_list[1].replace('♦', '')
    except:
        combination_product_size_list = combination.split('\n')
        combination_product_size_list1 = driver.find_element_by_class_name('std').text.split(':')
        combination = combination_product_size_list[0].replace('♦', '')
        
    #product_size = combination_product_size_list[4].replace('♦', '')
    
 # CATEGORY  *******************************************   
 
    try:
        category1 = product[1] #driver.find_element_by_class_name('category47').text
    except:
        category1 = ''
    try:
        category2 =  product[2]#driver.find_element_by_class_name('category52').text
    except:
        category2 = ''
    try:
        category3 = product[3]#driver.find_element_by_class_name('').text
    except:
        category3 = ''
    print('Cat1:', category1)
    print('Cat2:', category2)
    print('Cat3:', category3)

 # MATTER  *******************************************   
  
    mater = return_details('مواد التصنيع', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    if mater == '':
        mater = return_details('المواد الرئيسية', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    if mater == '':
        for  matter in combination_product_size_list1:
            if matter.find('Materials') == 0:
                
                mater = matter.replace('Materials', '').replace(':', '').replace('♦', '')

    print('Mater', mater)
# DETAIL  *******************************************   
    
    set_include = return_details('تفاصيل المنتج', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    description = return_details('تفاصيل المنتج', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    detail = return_details('حجم المنتج', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    if detail == '':
        for dt in combination_product_size_list1:


            if dt.find('Combination') == 0:
                print('coucou')
                detail = dt.replace('♦ ', '').replace('Combination', '').replace(':', '').replace('\n', '')
    print('Detail: ',detail)   
# COLOR  *******************************************   

    color = return_details('اللون', combination_product_size_list1).replace("\n", " ").replace('♦', '').replace(':', '')
    if color == '':
        # Additional
        try:
            additional_tabbed = driver.find_element_by_id('product_tabs_additional_tabbed')
            additional_tabbed.click()
            time.sleep(1)
            color = driver.find_element_by_id('product-attribute-specs-table').text.replace('اللون', '')
            
        except:
            color = ''
    print('Color: ', color)
# SIZE  *******************************************   
    size = return_details('حجم المنتج', combination_product_size_list1).replace("\n", " ").replace('♦', '').replace(':', '').replace('مصباح طاولة', '').replace('مصباح أرضي', '').replace('مصباح أرضي:', '').replace('ثريا سقف', '')
    
    
                
    
    if size.find('Table Lamp') == 0:
        size = size.replace('Table Lamp', '')
    if size == '':
        for size_find in combination_product_size_list1:
            if size_find.find('الحجم') == 0:
                size = size_find.replace('الحجم', '').replace(':', '').replace('\n', ''). replace('♦', '').replace('  ', '')
            elif size_find.find('Product Size') == 0:
                size = [toto for toto in combination_product_size_list1 if toto.find('Product Size') == 0][0]
    toto = size.split('*')
    print(toto)
    #Width
    try:
        width1 = [x for x in toto if 'عرض' in x or 'العرض' in x]
        width2 = width1[0].split(' ')
        width3 =  [x for x in width2 if x.isdigit()]
        width =  str(width3[0])
        print('width',width)
    except:
        print('Pas de width')
        width = ''
    #height
    try:
        height1 = [x for x in toto if 'إرتفاع' in x or ' ارتفاع'in x or 'الارتفاع' in x  ]
        height2 = height1[0].split(' ')
        height3 = [x for x in height2 if x.isdigit()]
        height = str(height3[0])
        print('height',height)
    except:
        print('Pas de height')
        height = ''

    #length
    try:
        length1 = [x for x in toto if 'عمق' in x or 'العمق' in x]
        length2 = length1[0].split(' ')
        length3 = [x for x in length2 if x.isdigit()]
        length = str(length3[0])
        print('length',length)
    except:
        print('Pas de height')
        length = ''
    
    if size == '':
        for size_find in combination_product_size_list1:
            if size_find.find('Product Size') == 0:
                pattern_H = 'H[1-9]+'
                pattern_D = 'D[1-9]+'
                pattern_W = 'W[1-9]+'
                width = [re.findall(pattern_W, com)[0].replace('W', '') for com in combination_product_size_list1 if re.findall(pattern_D, com) != []][0]
                height = [re.findall(pattern_H, com)[0].replace('H', '') for com in combination_product_size_list1 if re.findall(pattern_D, com) != []][0]
                length = [re.findall(pattern_D, com)[0].replace('D', '') for com in combination_product_size_list1 if re.findall(pattern_D, com) != []][0]

    
    
    
# STYLE  *******************************************   
    style = return_details('النوع :', combination_product_size_list1).replace("\n", " ").replace('♦', '')

    if style == '':
        try:
            for styl in combination_product_size_list1:
                if styl.find('Concept') == 0:
                    style = styl.replace('Concept :\n♦', '').replace(':', '').replace('\n', '')
        except:
            style = ''
    if style.find('نوع المنتج') == 0:
        style = style.replace('نوع المنتج', '').replace(':', '')
    print('Style:', style)
#     price = driver.find_element_by_id("product-attribute-specs-table").text
#     print(price)

# IMAGES ***************************************************
    list_images = [] 
    images = driver.find_elements_by_class_name("gallery-image")# 4 images.
    for i in images:
        list_images.append(i.get_attribute('src').split('?')[0])

    try:
        base_image =  list_images[0]
    except:
        base_image = ''
        
    try:
        additional_images = ",".join(list_images[1:])
    except:
        additional_images = ''
    #df['Images'] = list_images
    base_image = list_images[0]
    small_image = list_images[0]
    swatch_image = list_images[0]
    thumbnail_image = list_images[0]
    weight = 0
#*******************************************************
    two_month = datetime.now() + timedelta(days=60)
    two_month = two_month.strftime("%m/%d/%Y")
    today = datetime.today().strftime("%m/%d/%Y")
#*******************************************************
    series = pd.Series([sku,
                         name,
                         url_key,
                         detail,
                        description,
                        set_include,
                         color,
                         mater,
                        width,
                        height,
                        length,
                        weight,
                         base_image,
                         small_image,
                         swatch_image,
                         thumbnail_image,
                         additional_images ,
                         category1,
                         category2,
                         category3,
                         price ,
                        special_price,
                        'Catalog, Search',
                        'Taxable Goods',
                        link_url,
                        0,
                        -5,
                        in_stock,
                        1,
                        1,
                        'مستوردة',
                         today,
                         two_month,
                         'base',
                         'Default',
                         'simple',
                         '',
                        'DAS',
                        style

                        ],  index=df.columns)
    #df.append(df1)
    df = df.append(series, ignore_index=True)
    df.to_excel( f'{name_excel}', index = False)


In [6]:
combination_product_size_list1 = driver.find_element_by_class_name('std').text.split('\n\n')
combination_product_size_list1

['نظرة عامة : أليس ساحرا   أنه يمكنك اختيار مجموعة غرفة نوم عملية ولكن مع ذلك فهي أنيقة و متألقة. ',
 'الوصف : طقم غرفة نوم 6 قطع',
 'تفاصيل المنتج :\n♦ سرير 1 قطعة\n♦ كمودينو 2 قطعة\n♦ تسريحة 1 قطعة\n♦ مراية حائط 1 قطعة\n♦ خزانة ملابس ستة أبواب 1 قطعة',
 'حجم المنتج :\n♦ سرير * عرض 210 * عمق 193 * ارتفاع 121 سم\n♦ كمودينو * عرض 52 * عمق 40 * ارتفاع 47 سم\n♦ تسريحة * عرض 115 * عمق 40 * ارتفاع 81 سم\n♦ مراية حائط * عرض 102 * عمق 2 * ارتفاع 60 سم\n♦ خزانة ملابس ستة أبواب * عرض 240 * عمق 60 * ارتفاع 220 سم',
 'مواد التصنيع :\n♦ خشب صناعى مضغوط',
 'اللون : \n♦ بنى، بيج',
 'النوع : \n♦ معاصر',
 'إضغط هنا للوصول الى المرتبة المقترحة\n♦ (السعر غير شامل)']

In [5]:
driver.get('https://midasfurniture.com/ksa_ar/mela-bedroom-set-king-size-180-200-cm.html')

In [7]:
def return_details(value, list_dic):
        val = ''
        for i in list_dic:
            if value in i:
                val = i.replace(value, '')
        return val

In [10]:
detail = return_details('حجم المنتج', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
detail

'   سرير * عرض 210 * عمق 193 * ارتفاع 121 سم  كمودينو * عرض 52 * عمق 40 * ارتفاع 47 سم  تسريحة * عرض 115 * عمق 40 * ارتفاع 81 سم  مراية حائط * عرض 102 * عمق 2 * ارتفاع 60 سم  خزانة ملابس ستة أبواب * عرض 240 * عمق 60 * ارتفاع 220 سم'

# list accr

In [4]:
list_dic_cats = [
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sectional-sofa/page/', 'count': 5, 'cat1': ' أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' كنب زاوية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sofa-sets/page/', 'count': 7, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' أطقم كنب'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sofas-love-seats/page/', 'count': 4, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'كنب ثلاث مقاعد و مقعدين'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sofa-beds/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' كنب سرير'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/chairs/page/', 'count': 8, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' كراسي'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/bench-and-ottoman/page/', 'count': 8, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'بنش و أوتومان'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/center-side-tables/page/', 'count': 4, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'أطقم طاولات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/center-tables/page/', 'count': 7, 'cat1': 'أثاث منزلي', 'cat2': ' غرف المعيشة', 'cat3': ' طاولات وسط'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/end-tables/page/', 'count': 11, 'cat1': 'أثاث منزلي', 'cat2': ' غرف المعيشة', 'cat3': 'طاولات جانبية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/consoles/page/', 'count': 6, 'cat1': 'أثاث منزلي', 'cat2': ' غرف المعيشة', 'cat3': ' طاولات كونسول'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/nested-tables/page/', 'count': 3, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'طاولات متداخلة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/stands/page/', 'count': 3, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'ستاندات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/tv-units/page/', 'count': 7, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': ' مكتبات وطاولات تلفزيون'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/cabinets/page/', 'count': 4, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'خزائن'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/shoe-cabinets/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'خزائن الأحذية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/shelves/page/', 'count': 4, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'مكتبات ووحدات رفوف'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/chaise-lounge/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف المعيشة', 'cat3': 'شيزلونج'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/king-size-bedroom-sets/page/', 'count': 6, 'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'أطقم غرف النوم حجم ملكى'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/queen-size-bedroom-sets/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'أطقم غرف النوم حجم كوين'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/beds/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'سرير'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/night-stands/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': ' كومودينو'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/dresser/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'تسريحة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/chest-of-drawers/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'وحدة ادراج'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/wardrobes/page/', 'count': 3, 'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': ' خزانة ملابس'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/bedroom-sets/jewellery-chest/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف نوم', 'cat3': 'وحدة أدراج مجوهرات'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/dining-table-sets/page/', 'count': 5, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'أطقم غرف الطعام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/dining-tables/page/', 'count': 3, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'طاولة طعام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/dining-chairs/page/', 'count': 4, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'كراسي الطعام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/buffet/page/', 'count':3, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'بوفيه وقاعدة بوفيه'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/high-dining-table-sets/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'أطقم طاولات طعام مرتفعة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/high-stool-chairs/page/', 'count': 3, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'كراسي مرتفعة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/dining-rooms/counter-tables/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الطعام', 'cat3': 'طاولات متعددة الإستخدامات'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/recliners/rocker-recliner/page/', 'count': 4, 'cat1': 'أثاث منزلي', 'cat2': 'كراسي استرخاء', 'cat3': 'كرسي استرخاء هزاز'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/recliners/recliner/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': ' كراسي استرخاء', 'cat3': 'كرسي استرخاء'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/recliners/sofa-recliner/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'كراسي استرخاء', 'cat3': 'كنبة استرخاء'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/recliners/power-massage-recliner/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'كراسي استرخاء', 'cat3': 'كراسى استرخاء كهربائى و مساج'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/bedroom-set/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'أطقم غرف النوم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/beds/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': ' سرير'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/night-stands/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'كومودينو'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/dressers/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'تسريحة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/chest-of-drawers/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'وحدة أدراج'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/wardrobes/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'خزانة ملابس'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/study-desk-chairs/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'مكتب دراسة و كرسي'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/kids-room/bookcases-shelves/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'غرف الأطفال', 'cat3': 'مكتبات و أرفف'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/home-office/desks/page/', 'count': 4, 'cat1': 'أثاث منزلي', 'cat2': 'مكاتب منزلية', 'cat3': 'مكاتب'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/home-office/chairs/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'مكاتب منزلية', 'cat3': 'كراسي'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/super-king-size-200-200cm/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 200×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/super-king-size-193-203cm/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': ' فرشات مقاس 193×203 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/king-size-180-200cm/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 180×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/queen-size-150-200cm/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 150×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/queen-size-160x200cm/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': ' فرشات مقاس 160×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/full-size-135-190/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 135×190 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/twin-size-120-200cm/page/', 'count': 2, 'cat1': 'أثاث منزلي ', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 120×200 سم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/mattresses/single-size-90-100x190cm/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'فرشات السرير', 'cat3': 'فرشات مقاس 90 & 100 سم'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/gaming-chairs/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'طاولات وكراسى الألعاب', 'cat3': ''} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/pet-furniture/page/', 'count': 2, 'cat1': 'أثاث منزلي', 'cat2': 'أثاث الحيوانات الأليفة', 'cat3': ''} , 
{'url': 'https://midasfurniture.com/ksa_ar/home-furniture/outdoor-furniture/page/', 'count': 3, 'cat1': 'أثاث منزلي', 'cat2': 'أثاث خارجي', 'cat3': ''} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-chairs/high-back/page/', 'count': 2, 'cat1': 'أثاث مكتبي' , 'cat2': 'كراسي المكتب ', 'cat3': 'ظهر مرتفع' } , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-chairs/medium-back/page/', 'count': 2, 'cat1': 'أثاث مكتبي' , 'cat2': 'كراسي المكتب ', 'cat3':  'ظهر متوسط'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-chairs/low-back/page/', 'count': 2, 'cat1': 'أثاث مكتبي', 'cat2': 'كراسي المكتب', 'cat3': 'ظهر منخفض', } , 
    
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/visitor-chairs/page/', 'count': 3, 'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'كرسي زائر'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/desks-office-systems/page/', 'count': 4, 'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'مكاتب و مجموعة مكتبية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-sofas/page/', 'count': 3, 'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'كنبات المكتب'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/cabinets/page/', 'count': 2, 'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'خزائن'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/meeting-center-end-tables/page/', 'count': 3, 'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': ' طاولات الاجتماعات و الوسط و الطاولات الجانبية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/office-furniture/office-accessories/page/', 'count': 2, 'cat3': '', 'cat1': 'أثاث مكتبي', 'cat2': 'إكسسوارات المكاتب'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/dinner-sets.html/page/', 'count': 2, 'cat3': 'أطقم العشاء', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/dinner-and-serving-plates/page/', 'count': 4, 'cat3': 'صحون التقديم', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/bowls/page/', 'count': 5, 'cat3': 'أوعية تقديم', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/condiment-storage/page/', 'count': 2, 'cat3': 'تخزين البهارات', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/cups-and-saucers/page/', 'count': 4, 'cat3': 'فناجين وأطباق الشاى والقهوة', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/tea-and-sugar-pots/page/', 'count': 2, 'cat3': 'ابريق شاي وسكر', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/trays/page/', 'count': 4, 'cat3': 'الصواني', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/tiered-stand/page/', 'count': 3, 'cat3': 'حامل تقديم', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/baskets/page/', 'count': 2, 'cat3': 'سلة', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/jars/page/', 'count': 3, 'cat3': 'مرطبان', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/cutlery-sets/page/', 'count': 2, 'cat3': 'أدوات تناول الطعام', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/napkin-rings-and-holders/page/', 'count': 2, 'cat3': 'مسكة منديل', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/kitchen-and-dining/drinking-glasses-jugs/page/', 'count': 3, 'cat3': 'أواني زجاجية وأباريق', 'cat1': 'إكسسوارات', 'cat2': 'أدوات المطبخ'} , 

{'url': 'https://midasfurniture.com/ksa_ar/accessories/table-linen/table-runners/page/', 'count': 6, 'cat3': 'مفرش طاولة', 'cat1': 'إكسسوارات', 'cat2': 'مفرش طاولة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/table-linen/place-mats/page/', 'count': 7, 'cat3': 'حصيرة تحت الأطباق', 'cat1': 'إكسسوارات', 'cat2': 'مفرش طاولة'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/serving-trolleys/page/', 'count': 3, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'عربات التقديم'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/page/', 'count': 2, 'cat3': 'صحن صابون', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/soap-dispenser/page/', 'count': 2, 'cat3': 'موزع صابون', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/tumbler-and-tooth-brush-holders/page/', 'count': 2, 'cat3': 'حامل فرشاة الأسنان', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/toilet-paper-and-brush-holder/page/', 'count': 2, 'cat3': 'حامل ورق رول و فرشاة', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/towel-holder-and-racks/page/', 'count': 2, 'cat3': 'حامل مناشف', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/towels/page/', 'count': 2, 'cat3': 'منشفة', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/laundry-baskets/page/', 'count': 2, 'cat3': 'سلال غسيل', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/tissue-box/page/', 'count': 2, 'cat3': 'صندوق محارم', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/waste-bins/page/', 'count': 2, 'cat3': 'سلة مهملات', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/bathroom/bathroom-mats/page/', 'count': 2, 'cat3': 'سجاد الحمام', 'cat1': 'إكسسوارات', 'cat2': 'اكسسوارات الحمام'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/candle-holders/page/', 'count': 11, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'شمعدان'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/vases-and-flowers/page/', 'count': 11, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'مزهريات و أحواض نبات'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/artificial-flowers-and-trees/page/', 'count': 9, 'cat3': 'الزهور والنباتات', 'cat1': 'إكسسوارات', 'cat2': 'ورود ونباتات'} ,
{'url': 'https://midasfurniture.com/ksa_ar/accessories/artificial-flowers-and-trees/trees/page/', 'count': 2, 'cat3': 'الأشجار', 'cat1': 'إكسسوارات', 'cat2': 'ورود ونباتات'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/lighting/table-lamps/page/', 'count': 7, 'cat1': 'إكسسوارات', 'cat2': 'الإضاءة', 'cat3': 'أباجورة طاولة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/lighting/hanging-and-ceiling-lamps/page/', 'count': 4, 'cat1': 'إكسسوارات', 'cat2': 'الإضاءة', 'cat3': 'ثريا سقف'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/lighting/floor-lamps/page/', 'count': 2, 'cat1': 'إكسسوارات', 'cat2': 'الإضاءة', 'cat3': 'أباجورة أرضية'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/lighting/wall-lamp/page/', 'count': 2, 'cat1': 'إكسسوارات', 'cat2': 'الإضاءة', 'cat3': 'اباجورة حائط'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/photo-frames/page/', 'count': 3, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'براويز الصور'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/wall-arts-and-paintings/page/', 'count': 17, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'اللوحات وديكور الحائط'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/mirrors/page/', 'count': 7, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'المرايا'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/coat-hangers-screens/page/', 'count': 3, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'علاقات الملابس والفواصل'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/clocks/page/', 'count': 3, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'الساعات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/jewellery-organisers-box/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'صناديق للمجوهرات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/deco-figurines-and-objects/page/', 'count': 8, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'قطع فنية وديكورات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/ashtrays/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'منفضة سجائر'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/bedding-set/page/', 'count': 2, 'cat1': 'إكسسوارات', 'cat2': 'وسائد وأغطية', 'cat3': 'أطقم مفارش السرير'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/cushions/page/', 'count': 20, 'cat1': 'إكسسوارات', 'cat2': 'وسائد وأغطية', 'cat3': 'وسائد'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/throws/page/', 'count': 2, 'cat1': 'إكسسوارات', 'cat2': 'وسائد وأغطية', 'cat3': 'شال'} , 
{'url': 'https://midasfurniture.com/ksa_ar/accessories/soft-furnishings/sleeping-pillows.html', 'count': 2, 'cat1': 'إكسسوارات', 'cat2': 'وسائد وأغطية', 'cat3': 'مخدة سرير'} ,
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/doormats/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'دواسات الباب'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/accessories/safes/page/', 'count': 3, 'cat3': '', 'cat1': 'إكسسوارات', 'cat2': 'خزنات'} , 
    
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/mugs/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'فاخرة أكواب'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/cups-and-saucer-sets/page/', 'count': 3, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أطقم فناجين مع أطباق'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/cups-and-saucers/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'فناجين وأطباق'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/dinnerware/page/', 'count': 3, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أوانى طعام'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/drinking-dispenser/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'موزعات المشروبات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/drinking-glasses-sets/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أطقم أكواب المشروبات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/serving-bowls/page/', 'count': 3, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أوعية تقديم'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/condiment-storage/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أوعية تخزين البهارات'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/napkin-rings-and-holders/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'حامل مناديل المائدة'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/trays/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'صوانى'} , 
{'url': 'https://midasfurniture.com/ksa_ar/luxury-accessories/dinner-sets/page/', 'count': 2, 'cat3': '', 'cat1': 'إكسسوارات فاخرة', 'cat2': 'أطقم أوانى الطعام'} ,  
]

# Add categories

In [9]:
links_products = []

links_products = []
list_cat1 = []
list_cat2 = []
list_cat3 = []
for i, toto in enumerate(list_dic_cats):
    print('Count:', i)
    for i in range(1, toto['count']):
        url = f'{toto["url"]}{i}.html'
        driver.get(url)
        print(f'{toto["url"]}{i}.html')

        products = driver.find_elements_by_class_name('product-image')


        #Forloop to get all url products
        for product in products: 
            links_products.append(product.get_attribute('href'))
            list_cat1.append(toto['cat1'])
            list_cat2.append(toto['cat2'])
            list_cat3.append(toto['cat3'])
            time.sleep(0.5)
        time.sleep(1)
        print(len(links_products))
       
#Save link in excel files
link_living_rooms = {
    'liste': links_products,
    'cat1': list_cat1,
    'cat2': list_cat2,
    'cat3': list_cat3,
}
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
liste_link_living_rooms = pd.DataFrame(link_living_rooms)
liste_link_living_rooms.to_excel(f'link_all_product_midas_test-{dt_string}.xlsx')

Count: 0
https://midasfurniture.com/ksa_ar/home-furniture/living-rooms/sectional-sofa/page/1.html


KeyboardInterrupt: 

In [7]:

len(links_products)

NameError: name 'links_products' is not defined

# list product excel

In [3]:
list_product  = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/djangoSripte/media/Midas/urls/wafi-midas_update_products_url_.xlsx')

In [4]:
list_to_products = []

for index, row in list_product.iterrows():
    list_to_products.append( [row['url'], row['cat1'], row['cat2'], row['cat3'] ] , )

In [5]:
len(list_product)

6605

In [42]:
combination = driver.find_element_by_xpath('//div[@itemprop="description"]').text
combination

'نظرة عامة : امتلك الراحة مع هذه الأريكة القابلة للاستلقاء وقم بتعزيز تجربة الجلوس الخاصة بك. لتذهب في طريق الاسترخاء اللانهائي بضغطة واحدة فقط.\n\nوصف المنتج : طقم كنبة زاوية ثلاثة قطع\n\nتفاصيل المنتج :\n♦ كنبة 3مقعد راحة 1 قطعة\n♦ كنبة 2مقعد راحة 1 قطعة\n♦ كنبة زاوية مقعد واحد 1 قطعة\n\nحجم المنتج :\n♦ كنبة 3مقعد راحة * عرض 221 * عمق 101 * ارتفاع 106 سم\n♦ كنبة 2مقعد راحة * عرض 203 * عمق 101 * ارتفاع 106 سم\n♦ كنبة زاوية مقعد واحد * عرض 175 * عمق 101 * ارتفاع 106 سم\n\nالأقمشة :\n♦ بوليستر\n\nاللون :\n♦ رمادى داكن\n\nالنوع :\n♦ إنتقالى'

In [49]:
driver.get(list_to_products[0][0])

In [51]:
driver.get(str(list_to_products[0][0]))

In [50]:
list_to_products[0][0]

'https://midasfurniture.com/ksa_ar/mccade-sectional-sofa.html'

In [57]:


name = driver.find_element_by_xpath('//h1[@class="page-title"]').text.strip()

link_url = str(product[0])

in_stock = driver.find_element_by_class_name('stock').text

sku = driver.find_element_by_class_name('sku').text[4:]
sku = 'DAS-' + sku 
url_key = (f'{sku}-{name}').replace(',', '-').replace('*', '-').replace('،', '-').replace('/', '-').replace('%', '').replace('"', '')


special_price =  ''

price = driver.find_element_by_xpath('//span[@class="price"]').text.replace('٫', '.').replace('٬', '').replace('SAR', '').strip()

combination = driver.find_element_by_xpath('//div[@itemprop="description"]').text
price


'10500.00'

In [10]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/djangoSripte/media/Midas/midas_update_products_url_28_09_2021v2.xlsx')

In [11]:
df.shape

(6913, 6)

In [207]:
list_to_products = []
for index, row in toto.iterrows():
    list_to_products.append([
        row['Link_url'],
        row['categories1'],
         row['categories2'],
        row['categories3'],
        ])

In [208]:
list_to_products

[['https://midasfurniture.com/ksa_ar/elephant-stool.html',
  'أثاث منزلي',
  'غرف المعيشة',
  'بنش و أوتومان'],
 ['https://midasfurniture.com/ksa_ar/rumano-coffee-table-set-3-pcs.html',
  'أثاث منزلي',
  'غرف المعيشة',
  'أطقم طاولات'],
 ['https://midasfurniture.com/ksa_ar/alabix-center-table-set-3-pcs.html',
  'أثاث منزلي',
  'غرف المعيشة',
  'أطقم طاولات'],
 ['https://midasfurniture.com/ksa_ar/fardeen-center-table.html',
  'أثاث منزلي',
  ' غرف المعيشة',
  ' طاولات وسط'],
 ['https://midasfurniture.com/ksa_ar/unicorn-center-table.html',
  'أثاث منزلي',
  ' غرف المعيشة',
  ' طاولات وسط'],
 ['https://midasfurniture.com/ksa_ar/goldsmith-center-table-16038.html',
  'أثاث منزلي',
  ' غرف المعيشة',
  ' طاولات وسط'],
 ['https://midasfurniture.com/ksa_ar/leistra-shelf-small.html',
  'أثاث منزلي',
  'غرف المعيشة',
  'مكتبات ووحدات رفوف'],
 ['https://midasfurniture.com/ksa_ar/ronto-shelf.html',
  'أثاث منزلي',
  'غرف المعيشة',
  'مكتبات ووحدات رفوف'],
 ['https://midasfurniture.com/ksa_ar/arcvel

In [204]:
toto = df.loc[df['price']==0]

In [205]:
toto

,sku,name,url_key,short_description,description,set_include,free_colors,raw_materials,ts_dimensions_width,ts_dimensions_height,...,attribute_set_code,product_type,store_view_code,supplier,style,First,Last,estimated_delivery_text,estimated_delivery_enable,sku number only
616,DAS-119539,اوتومان ايليفانت,اوتومان ايليفانت-DAS-119539,اوتومان 1 قطعة,اوتومان * عرض 47 * عمق 29 * ارتفاع 39 سم,اوتومان 1 قطعة,بنى/ بيج,ريسين,47.0,39.0,...,Default,simple,NaN,DAS,معاصر,اوتومان ايليفانت,None,,Static Text,119539
696,DAS-127410,طقم طاولات قهوة 3 قطع رومانو,طقم طاولات قهوة 3 قطع رومانو-DAS-127410,طاولة وسط 1 قطعة طاولة جانبيه 2 قطعة,طاولة وسط * عرض 127 * عمق 76 * ارتفاع 46 سم...,طاولة وسط 1 قطعة طاولة جانبيه 2 قطعة,فضى,معدن / زجاج,127.0,46.0,...,Default,simple,NaN,DAS,معاصر,طقم طاولات قهوة 3 قطع رومانو,None,,Static Text,127410
698,DAS-124189,طقم طاولات وسط 3 قطع ألابيكس,طقم طاولات وسط 3 قطع ألابيكس-DAS-124189,طاولة وسط 1 قطعة طاولة جانبيه 2 قطعة,طاولة وسط * عرض 84 * عمق 84 * ارتفاع 46 سم ...,طاولة وسط 1 قطعة طاولة جانبيه 2 قطعة,شفاف، أسود,خشب مصنع / معدن / زجاج,84.0,46.0,...,Default,simple,NaN,DAS,كلاسيكى,طقم طاولات وسط 3 قطع ألابيكس,None,,Static Text,124189
755,DAS-126744,طاولة وسط فارديين,طاولة وسط فارديين-DAS-126744,طاولة وسط 1 قطعة,طاولة وسط * عرض 200 * عمق 100 * ارتفاع 45 سم,طاولة وسط 1 قطعة,فضى,خشب مصنع,200.0,45.0,...,Default,simple,NaN,DAS,معاصر,طاولة وسط فارديين,None,,Static Text,126744
756,DAS-126743,طاولة وسط يونيكورن,طاولة وسط يونيكورن-DAS-126743,طاولة وسط عدد 1 قطعة,طاولة وسط العرض 150 * العمق 85 * الارتفاع ...,طاولة وسط عدد 1 قطعة,فضى,NaN,150.0,45.0,...,Default,simple,NaN,DAS,معاصر,طاولة وسط يونيكورن,None,,Static Text,126743
757,DAS-126694,طاولة وسط جولد سميث,طاولة وسط جولد سميث-DAS-126694,طاولة وسط 1 قطعة,طاولة وسط * عرض 200 * عمق 100 * ارتفاع 45 سم,طاولة وسط 1 قطعة,ذهبى,NaN,200.0,45.0,...,Default,simple,NaN,DAS,معاصر,طاولة وسط جولد سميث,None,,Static Text,126694
1550,DAS-113441,مكتبة ارفف صغيرة للكتب ليسترا,مكتبة ارفف صغيرة للكتب ليسترا-DAS-113441,مكتبة ارفف للكتب 1 قطعة,مكتبة ارفف للكتب * عرض 45 * عمق 35 * ارتفاع...,مكتبة ارفف للكتب 1 قطعة,NaN,NaN,45.0,99.0,...,Default,simple,NaN,DAS,NaN,مكتبة ارفف صغيرة للكتب ليسترا,None,,Static Text,113441
1551,DAS-113002,مكتبة ارفف رونتو,مكتبة ارفف رونتو-DAS-113002,مكتبة ارفف متعددة الإستخدامات 1 قطعة,مكتبة ارفف متعددة الإستخدامات * عرض 40 * عم...,مكتبة ارفف متعددة الإستخدامات 1 قطعة,بنى فاتح,خشب,40.0,76.0,...,Default,simple,NaN,DAS,معاصر,مكتبة ارفف رونتو,None,,Static Text,113002
1637,DAS-115913,طقم غرفة نوم سرير ملكى اركفيلور,طقم غرفة نوم سرير ملكى اركفيلور-DAS-115913,سرير 1 قطعة كمودينو 2 قطعة تسريحة 1 قطعة ...,سرير * عرض 196 * عمق 216 * ارتفاع 160 سم ك...,سرير 1 قطعة كمودينو 2 قطعة تسريحة 1 قطعة ...,فضى / بيج,خشب صلب / خشب مصنع,196.0,160.0,...,Default,simple,NaN,DAS,معاصر,طقم غرفة نوم سرير ملكى اركفيلور,None,,Static Text,115913
1638,DAS-115861,سيرافيك طقم غرفة نوم سرير ملكي,سيرافيك طقم غرفة نوم سرير ملكي-DAS-115861,NaN,سرير ملكي عرض 193 * عمق 203 * ارتفاع 160 سم...,NaN,NaN,NaN,193.0,160.0,...,Default,simple,NaN,DAS,NaN,سيرافيك طقم غرفة نوم سرير ملكي,None,,Static Text,115861


In [160]:
list_urls = []

In [158]:
for index, row in toto.iterrows():
    list_urls.append([row['Link_url']])

['https://midasfurniture.com/ksa_ar/elephant-stool.html',
 'https://midasfurniture.com/ksa_ar/rumano-coffee-table-set-3-pcs.html',
 'https://midasfurniture.com/ksa_ar/alabix-center-table-set-3-pcs.html',
 'https://midasfurniture.com/ksa_ar/fardeen-center-table.html',
 'https://midasfurniture.com/ksa_ar/unicorn-center-table.html',
 'https://midasfurniture.com/ksa_ar/goldsmith-center-table-16038.html',
 'https://midasfurniture.com/ksa_ar/leistra-shelf-small.html',
 'https://midasfurniture.com/ksa_ar/ronto-shelf.html',
 'https://midasfurniture.com/ksa_ar/arcvelour-bedroom-set-7-pcs.html',
 'https://midasfurniture.com/ksa_ar/glamorgan.html',
 'https://midasfurniture.com/ksa_ar/caitbrook-bedroom-set-king-size-193-203-cm.html',
 'https://midasfurniture.com/ksa_ar/mangual-bed-king-size-193-203-cm.html',
 'https://midasfurniture.com/ksa_ar/viabella-headboard-footboard.html',
 'https://midasfurniture.com/ksa_ar/aride-office-visitor-chair.html',
 'https://midasfurniture.com/ksa_ar/dolfa-office-v

In [141]:
name = driver.find_element_by_xpath('//h1[@class="page-title"]').text.strip()

link_url = str(product[0])

in_stock = driver.find_element_by_class_name('stock').text

sku = driver.find_element_by_class_name('sku').text[4:]
sku = 'DAS-' + sku 
url_key = (f'{sku}-{name}').replace(',', '-').replace('*', '-').replace('،', '-').replace('/', '-').replace('%', '').replace('"', '')


special_price = driver.find_elements_by_class_name('price')[0].get_attribute('innerHTML').replace('SAR', '').replace('٫', '.').replace(' ', '')

price = driver.find_elements_by_class_name('price')[1].get_attribute('innerHTML').replace('SAR', '').replace('٫', '.').replace(' ', '')

combination = driver.find_element_by_xpath('//div[@itemprop="description"]').text
name, sku, special_price, price, in_stock

('كنبة إسترخاء ميشينر',
 'DAS-13824',
 '3٬181.75&nbsp;',
 '4٬895.00&nbsp;',
 'متوفر')

In [223]:
images = driver.find_elements_by_xpath('//div[@class="fotorama__nav-wrap fotorama__nav-wrap--horizontal"]//img')
print('Len: ',len(images))
if len(images) >= 5:
    images = images[:5]

    list_images1 = []
    for img in images:
        mimi = extract_images()
        aa = img.get_attribute('src').replace('https://midasfurniture.com/media/catalog/product/cache/', '').split('/')
        aa[0] = mimi
        list_images1.append('/'.join(aa))
    list_images = ['https://midasfurniture.com/media/catalog/product/cache/' + ig for ig in list_images1]
elif len(images) == 0:
    list_images = []
    print('ppap')
    imagtoto = driver.find_element_by_xpath('//img[@class="fotorama__img"]')
    list_images.append(imagtoto.get_attribute('src'))
list_images

Len:  9


['https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/124584_g.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/124584_f.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/124584_d.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/124584_i.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/2/124584_h.jpg']

In [215]:
imagtoto = driver.find_element_by_xpath('//img[@class="fotorama__img"]')
imagtoto.get_attribute('src')
# print(imagtoto.get_attribute('innerHTML'))

'https://midasfurniture.com/media/catalog/product/cache/7bd35ba35992fa136f6e5c8a3779eb8b/4/2/421115_791x633_00.jpg'

In [160]:
list_images

['https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_c.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_b.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_c.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_d.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_f.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_h.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_j.jpg',
 'https://midasfurniture.com/media/catalog/product/cache/ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_l.jpg']

In [178]:

def extract_images():
    text = driver.find_element_by_class_name('zoomWindowContainer').get_attribute('innerHTML')
    regex = re.compile('(http(s?):)([/|.|\w|\s|-])*\.(?:jpg|gif|png)')
    imgs = re.search(regex, text)
    if imgs != None:
        
        return imgs[0].replace('https://midasfurniture.com/media/catalog/product/cache/', '').split('/')[0]
    else:
        return ''


In [207]:
extract_images()

'ce18d1d07b138f93805957ee70f15be2'

In [168]:
titi.split('/cache/')

['https://midasfurniture.com/media/catalog/product',
 'ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_l.jpg']

In [170]:
titch = 'ce18d1d07b138f93805957ee70f15be2/1/1/11-91285_l.jpg'
titch.split('/')[0]

'ce18d1d07b138f93805957ee70f15be2'

# MaIN FONCTION

In [6]:
def urlimage(text):
    '''finds image url'''
    imageurl = []
    imageurl = re.findall(r'https?:\/\/.*\.(?:png|jpg)', text)
    return imageurl

def extract_url_images(driver):
    r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    time.sleep(0.5)
    soup = BeautifulSoup(r, 'html.parser')
#     images_base = soup.find('div', {'class': 'zoomWindow'})
#     images_base = urlimage(str(images_base))[0]
#     images_base = images_base.replace('https://midasfurniture.com/media/catalog/product/cache/', '')
#     code_produit = images_base.split('/')[0]
    code_produit = 'ce18d1d07b138f93805957ee70f15be2'
    print('Code: ', code_produit)
    list_images = []
    
    images = soup.find_all('img', {'class': 'fotorama__img'})
    len(images)
    if len(images)>=8:
        images = images[:6]
    for img in images:
        url_toto1 = img['src']
        titit = url_toto1.replace('https://midasfurniture.com/media/catalog/product/cache/', '')
        titit = '/'.join(titit.split('/')[1: ])

        list_images.append('https://midasfurniture.com/media/catalog/product/cache/' + code_produit +'/'+ titit)
    return list_images


def extract_element(name, soup):
    regex = re.compile(f'{name}')
    result = None
    for elem in soup(text=regex):
        
        try:
            result = elem.next_element.next_element.replace('♦', '').strip()
        except:
            try:
                result = elem.next_element.next_element.next_element.replace('♦', '').strip()
            except:
                result = None
    return  result


def extract_images():
    text = driver.find_element_by_class_name('zoomWindowContainer').get_attribute('innerHTML')
    regex = re.compile('(http(s?):)([/|.|\w|\s|-])*\.(?:jpg|gif|png)')
    imgs = re.search(regex, text)
    if imgs != None:
        
        return imgs[0].replace('https://midasfurniture.com/media/catalog/product/cache/', '').split('/')[0]
    else:
        return ''

def return_details(value, list_dic):
        val = ''
        for i in list_dic:
            if value in i:
                val = i.replace(value, '')
        return val
count = 0
name_excel = 'midas_update_19-07-22-5.xlsx'
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/Categories/midas1_model.xlsx')
#ForLoop main
for i, product in enumerate(list_to_products[4001: 5000]):
#for i, product in enumerate(list_to_products):
    time.sleep(1)
    try:
        print('*************************************')
        print('Count:', i)
        print('URL:', product[0])
        if '/ksa_ar/' in product[0]:
            print('It is ok')
        driver.get('https://midasfurniture.com/ksa_ar/mccade-sectional-sofa.html')
        driver.get(product[0])
        time.sleep(5)
    except:
        continue
    try:

        name = driver.find_element_by_xpath('//h1[@class="page-title"]').text.strip()
    
        link_url = str(product[0])

        in_stock = driver.find_element_by_class_name('stock').text

        sku = driver.find_element_by_class_name('sku').text[4:]
        sku = 'DAS-' + sku 
        url_key = (f'{sku}-{name}').replace(',', '-').replace('*', '-').replace('،', '-').replace('/', '-').replace('%', '').replace('"', '')
    
        
        special_price =  ''

        price = driver.find_element_by_xpath('//span[@class="price"]').text.replace('٫', '.').replace('٬', '').replace('SAR', '').strip()

        combination = driver.find_element_by_xpath('//div[@itemprop="description"]').text
    except:
        print('Error combination...')
        continue
    try:
        combination_product_size_list = combination.split('\n')
        combination_product_size_list1 = driver.find_element_by_xpath('//div[@itemprop="description"]').text.split('\n\n')
        combination = combination_product_size_list[1].replace('♦', '')
    except:
        combination_product_size_list = combination.split('\n')
        combination_product_size_list1 = driver.find_element_by_xpath('//div[@itemprop="description"]').text.split(':')
        combination = combination_product_size_list[0].replace('♦', '')
        
    #product_size = combination_product_size_list[4].replace('♦', '')
    
 # CATEGORY  *******************************************   
 
    try:
        category1 = product[1] #driver.find_element_by_class_name('category47').text
    except:
        category1 = ''
    try:
        category2 =  product[2]#driver.find_element_by_class_name('category52').text
    except:
        category2 = ''
    try:
        category3 = product[3]#driver.find_element_by_class_name('').text
    except:
        category3 = ''
    print('Cat1:', category1)
    print('Cat2:', category2)
    print('Cat3:', category3)

 # MATTER  *******************************************   
  
    mater = return_details('مواد التصنيع', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    if mater == '':
        mater = return_details('المواد الرئيسية', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    if mater == '':
        for  matter in combination_product_size_list1:
            if matter.find('Materials') == 0:
                
                mater = matter.replace('Materials', '').replace(':', '').replace('♦', '')
    
    print('Mater', mater)
    
    r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    soup = BeautifulSoup(r, 'html.parser')

    def extract_elem(name, soup):
        try:
            return str(soup.find('b', text=re.compile(name)).next_element.next_element.next_element).replace('♦', '').strip()
        except:
            return ''
    raw_material = extract_elem('مواد التصنيع :', soup)
    free_color1 = extract_elem('اللون :', soup)
    style1 = extract_elem('النوع :', soup)
#     no_of_peices = extract_elem('الوصف :', soup)
    sentence = [  'الوصف ', 'وصف المنتج',]
    no_of_peices = ''
    for sen in sentence:
        try:
            try:
                no_of_peices = soup.find('strong', text=re.compile(sen)).next_element.next_element.strip()
                break
            except:
                no_of_peices = soup.find('b', text=re.compile(sen)).next_element.next_element.strip()
                break
        except:
            continue
    try:
        firstElem = soup.find('', text=re.compile('حجم المنتج'))

        if not firstElem:
            firstElem = soup.find('', text=re.compile('الحجم'))
        lastElem = soup.find('', text=re.compile('مواد التصنيع'))
        if not lastElem:
            lastElem = soup.find('', text=re.compile('اللون'))
        products_size1 = []
        next = firstElem.next_element
        while next != lastElem:

            if '♦' in next:
                print('next: ', next)
                products_size1.append(next)
            next = next.next_element
    except:
        products_size1 = []
# DETAIL  *******************************************   
    
    detail = return_details('تفاصيل المنتج', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    description = return_details('حجم المنتج', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    set_include = return_details('تفاصيل المنتج', combination_product_size_list1).replace("\n", " ").replace(':', '').replace('♦', '')
    if detail == '':
        for dt in combination_product_size_list1:


            if dt.find('Combination') == 0:
                print('coucou')
                detail = dt.replace('♦ ', '').replace('Combination', '').replace(':', '').replace('\n', '')
    print('Detail: ',detail)   
# COLOR  *******************************************   

    color = return_details('اللون', combination_product_size_list1).replace("\n", " ").replace('♦', '').replace(':', '')
    if color == '':
        # Additional
        try:
            additional_tabbed = driver.find_element_by_id('product_tabs_additional_tabbed')
            additional_tabbed.click()
            time.sleep(1)
            color = driver.find_element_by_id('product-attribute-specs-table').text.replace('اللون', '')
            
        except:
            color = ''
    print('Color: ', color)
# SIZE  *******************************************   
    size = return_details('حجم المنتج', combination_product_size_list1).replace("\n", " ").replace('♦', '').replace(':', '').replace('مصباح طاولة', '').replace('مصباح أرضي', '').replace('مصباح أرضي:', '').replace('ثريا سقف', '')
    
    
                
    
    if size.find('Table Lamp') == 0:
        size = size.replace('Table Lamp', '')
    if size == '':
        for size_find in combination_product_size_list1:
            if size_find.find('الحجم') == 0:
                size = size_find.replace('الحجم', '').replace(':', '').replace('\n', ''). replace('♦', '').replace('  ', '')
            elif size_find.find('Product Size') == 0:
                size = [toto for toto in combination_product_size_list1 if toto.find('Product Size') == 0][0]
    toto = size.split('*')
    print(toto)
    #Width
    try:
        width1 = [x for x in toto if 'عرض' in x or 'العرض' in x]
        width2 = width1[0].split(' ')
        width3 =  [x for x in width2 if x.isdigit()]
        width =  str(width3[0])
        print('width',width)
    except:
        print('Pas de width')
        width = ''
    #height
    try:
        height1 = [x for x in toto if 'إرتفاع' in x or ' ارتفاع'in x or 'الارتفاع' in x  ]
        height2 = height1[0].split(' ')
        height3 = [x for x in height2 if x.isdigit()]
        height = str(height3[0])
        print('height',height)
    except:
        print('Pas de height')
        height = ''

    #length
    try:
        length1 = [x for x in toto if 'عمق' in x or 'العمق' in x]
        length2 = length1[0].split(' ')
        length3 = [x for x in length2 if x.isdigit()]
        length = str(length3[0])
        print('length',length)
    except:
        print('Pas de height')
        length = ''
    
    if size == '':
        for size_find in combination_product_size_list1:
            if size_find.find('Product Size') == 0:
                pattern_H = 'H[1-9]+'
                pattern_D = 'D[1-9]+'
                pattern_W = 'W[1-9]+'
                width = [re.findall(pattern_W, com)[0].replace('W', '') for com in combination_product_size_list1 if re.findall(pattern_D, com) != []][0]
                height = [re.findall(pattern_H, com)[0].replace('H', '') for com in combination_product_size_list1 if re.findall(pattern_D, com) != []][0]
                length = [re.findall(pattern_D, com)[0].replace('D', '') for com in combination_product_size_list1 if re.findall(pattern_D, com) != []][0]

    
    
    
# STYLE  *******************************************   
    style = return_details('النوع :', combination_product_size_list1).replace("\n", " ").replace('♦', '')

    if style == '':
        try:
            for styl in combination_product_size_list1:
                if styl.find('Concept') == 0:
                    style = styl.replace('Concept :\n♦', '').replace(':', '').replace('\n', '')
        except:
            style = ''
    if style.find('نوع المنتج') == 0:
        style = style.replace('نوع المنتج', '').replace(':', '')
    print('Style:', style)
#     price = driver.find_element_by_id("product-attribute-specs-table").text
#     print(price)
    estimated_delivery_text = driver.find_element_by_class_name('product-info-deliverydays').text
    
    r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    soup = BeautifulSoup(r, 'html.parser')
    description = soup.find('div', {'itemprop': 'description'}).text.split('المنتج :')
    products_size = ''
    for des in description:
        if 'حجم' in des:
            products_size = des.replace('حجم', '')
    free_colors = extract_element('اللون', soup)
    style = extract_element('النوع', soup)
    # IMAGES ***************************************************
    try:
        tita = '...'
        driver.find_element_by_xpath(f'//img[@src="{tita}"]').click()
    except:
        pass
    list_images = extract_url_images(driver)
    list_images = list(dict.fromkeys(list_images))
    
    base_image =  list_images[0]
        
    try:
        additional_images = ",".join(list_images[1:])
    except:
        additional_images = ''
    #df['Images'] = list_images
    base_image = list_images[0]
    small_image = list_images[0]
    swatch_image = list_images[0]
    thumbnail_image = list_images[0]
    weight = 0
#*******************************************************
    two_month = datetime.now() + timedelta(days=60)
    two_month = two_month.strftime("%m/%d/%Y")
    today = datetime.today().strftime("%m/%d/%Y")
#*******************************************************
#     series = pd.Series([sku,
#                          name,
#                          url_key,
#                          detail,
#                         description,
#                         set_include,
#                          free_colors,
#                          mater,
#                         products_size,
#                         width,
#                         height,
#                         length,
#                         weight,
#                          base_image,
#                          small_image,
#                          swatch_image,
#                          thumbnail_image,
#                          additional_images ,
#                          category1,
#                          category2,
#                          category3,
#                          price ,
#                         special_price,
#                         'Catalog, Search',
#                         'Taxable Goods',
#                         link_url,
#                         0,
#                         -5,
#                         in_stock,
#                         1,
#                         1,
#                         'مستوردة',
#                          today,
#                          two_month,
#                          'base',
#                          'Default',
#                          'simple',
#                          '',
#                         'DAS',
#                         style, 
#                         estimated_delivery_text,

#                         ],  index=df.columns)
    #df.append(df1)
    
    data = {
        'sku': sku,
        'name': name,
        'url_key': url_key,
        'detail': detail,
        'description': description,
        'set_include': set_include,
        'free_colors': free_colors,
        'mater': mater,
        'products_size': products_size,
        'width': width,
        'height': height,
        'length': length,
        'weight': weight,
        'base_image': base_image,
        'small_image': small_image,
        'swatch_image': swatch_image,
        'thumbnail_image': thumbnail_image,
        'additional_images': additional_images,
        'category1': category1,
        'category2': category2,
        'category3': category3,
        'price': price,
        'special_price': special_price,
        'visibility': 'Catalog, Search',
        'tax_class_name' : 'Taxable Goods',
        'Link_url': link_url,
        'qty': 0,
        'out_of_stock_qty': -5,
        'is_in_stock': in_stock,
        'product_online': 1,
        'allow_backorders': 1,
        'manufacturer': 'مستوردة',
         'news_from_date': today,
         'news_to_date': two_month,
         'product_websites': 'base',
         'attribute_set_code': 'Default',
         'product_type': 'simple',
         'store_view_code': '',
        'supplier': 'DAS',
        'style': style, 
        'estimated_delivery_text': estimated_delivery_text,
        'raw_material': raw_material,
        'free_color1': free_color1,
        'style1': style1,
        'no_of_peices': no_of_peices,
        'product_size1': products_size1,
    }
    #df = df.append(series, ignore_index=True)
    df1 = pd.DataFrame([data])
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel( f'{name_excel}', index = False)


*************************************
Count: 0
URL: https://midasfurniture.com/ksa_ar/vogesite-planter-12752.html
It is ok
Cat1: إكسسوارات
Cat2: مزهريات و أحواض نبات
Cat3: nan
Mater حجم المنتج   إصيص نباتات * عرض 33 * عمق 62 * ارتفاع 90 سم    ألياف زجاجية اللون   ذهبى النوع   معاصر
next:  ♦ إصيص نباتات * عرض 33 * عمق 62 * ارتفاع 90 سم
Detail:  نظرة عامة  اذا كان منزلك يحتاج إلى لمسة من التفاصيل البيضاء فكل ما عليك فعله هو أن تلق نظرة على تشكيلة اكسسوارات المطبخ لدينا وصف المنتج  إصيص نباتات    إصيص نباتات 1 قطعة
Color:  حجم المنتج   إصيص نباتات * عرض 33 * عمق 62 * ارتفاع 90 سم مواد التصنيع   ألياف زجاجية    ذهبى النوع   معاصر
['   إصيص نباتات ', ' عرض 33 ', ' عمق 62 ', ' ارتفاع 90 سم مواد التصنيع   ألياف زجاجية اللون   ذهبى النوع   معاصر']
width 33
height 90
length 62
Style: حجم المنتج :  إصيص نباتات * عرض 33 * عمق 62 * ارتفاع 90 سم مواد التصنيع :  ألياف زجاجية اللون :  ذهبى   معاصر
Code:  ce18d1d07b138f93805957ee70f15be2
*************************************
Count: 1
URL: https://mida

next:  ♦ إصيص نباتات * عرض 40 * عمق 40 * ارتفاع 110 سم
Detail:  وصف المنتج  إصيص نباتات    إصيص نباتات 1 قطعة
Color:  حجم المنتج   إصيص نباتات * عرض 40 * عمق 40 * ارتفاع 110 سم مواد التصنيع   ألياف زجاجية    أبيض، فضى النوع   معاصر
['   إصيص نباتات ', ' عرض 40 ', ' عمق 40 ', ' ارتفاع 110 سم مواد التصنيع   ألياف زجاجية اللون   أبيض، فضى النوع   معاصر']
width 40
height 110
length 40
Style: حجم المنتج :  إصيص نباتات * عرض 40 * عمق 40 * ارتفاع 110 سم مواد التصنيع :  ألياف زجاجية اللون :  أبيض، فضى   معاصر
Code:  ce18d1d07b138f93805957ee70f15be2
*************************************
Count: 11
URL: https://midasfurniture.com/ksa_ar/eilstra-planter.html
It is ok
Cat1: إكسسوارات
Cat2: مزهريات و أحواض نبات
Cat3: nan
Mater حجم المنتج   إصيص نباتات * عرض 26 * عمق 26 * ارتفاع 99 سم    ألياف زجاجية اللون   أبيض النوع   معاصر
next:  ♦ إصيص نباتات * عرض 26 * عمق 26 * ارتفاع 99 سم
Detail:  وصف المنتج  إصيص نباتات    إصيص نباتات 1 قطعة
Color:  حجم المنتج   إصيص نباتات * عرض 26 * عمق 26 * ارتفاع 99 سم م

Code:  ce18d1d07b138f93805957ee70f15be2
*************************************
Count: 23
URL: https://midasfurniture.com/ksa_ar/covite-planter.html
It is ok
Cat1: إكسسوارات
Cat2: مزهريات و أحواض نبات
Cat3: nan
Mater حجم المنتج   إصيص نباتات * عرض 41 * عمق 41 * ارتفاع 71 سم    ألياف زجاجية اللون   أبيض النوع   معاصر
next:  ♦ إصيص نباتات * عرض 41 * عمق 41 * ارتفاع 71 سم
Detail:  وصف المنتج  إصيص نباتات    إصيص نباتات 1 قطعة
Color:  حجم المنتج   إصيص نباتات * عرض 41 * عمق 41 * ارتفاع 71 سم مواد التصنيع   ألياف زجاجية    أبيض النوع   معاصر
['   إصيص نباتات ', ' عرض 41 ', ' عمق 41 ', ' ارتفاع 71 سم مواد التصنيع   ألياف زجاجية اللون   أبيض النوع   معاصر']
width 41
height 71
length 41
Style: حجم المنتج :  إصيص نباتات * عرض 41 * عمق 41 * ارتفاع 71 سم مواد التصنيع :  ألياف زجاجية اللون :  أبيض   معاصر
Code:  ce18d1d07b138f93805957ee70f15be2
*************************************
Count: 24
URL: https://midasfurniture.com/ksa_ar/alnoite-planter.html
It is ok
Cat1: إكسسوارات
Cat2: مزهريات و أحواض نب

Code:  ce18d1d07b138f93805957ee70f15be2
*************************************
Count: 34
URL: https://midasfurniture.com/ksa_ar/elwood-vase-set-of-3-pcs.html
It is ok
Cat1: إكسسوارات
Cat2: مزهريات و أحواض نبات
Cat3: nan
Mater حجم المنتج   مزهرية * عرض 8 * عمق 8 * ارتفاع 36 سم  مزهرية * عرض 8 * عمق 8 * ارتفاع 25 سم  مزهرية * عرض 8 * عمق 8 * ارتفاع 15 سم    سيراميك اللون   أخضر فاتح، رمادى النوع   إنتقالى
next:  ♦ مزهرية * عرض 8 * عمق 8 * ارتفاع 36 سم
next:  ♦ مزهرية * عرض 8 * عمق 8 * ارتفاع 25 سم
next:  ♦ مزهرية * عرض 8 * عمق 8 * ارتفاع 15 سم
Detail:  وصف المنتج  طقم مزهريات 3 قطع    مزهرية 1 قطعة  مزهرية 1 قطعة  مزهرية 1 قطعة
Color:  حجم المنتج   مزهرية * عرض 8 * عمق 8 * ارتفاع 36 سم  مزهرية * عرض 8 * عمق 8 * ارتفاع 25 سم  مزهرية * عرض 8 * عمق 8 * ارتفاع 15 سم مواد التصنيع   سيراميك    أخضر فاتح، رمادى النوع   إنتقالى
['   مزهرية ', ' عرض 8 ', ' عمق 8 ', ' ارتفاع 36 سم  مزهرية ', ' عرض 8 ', ' عمق 8 ', ' ارتفاع 25 سم  مزهرية ', ' عرض 8 ', ' عمق 8 ', ' ارتفاع 15 سم مواد التصنيع   سيراميك 

Code:  ce18d1d07b138f93805957ee70f15be2
*************************************
Count: 43
URL: https://midasfurniture.com/ksa_ar/aplite-planter-small.html
It is ok
Cat1: إكسسوارات
Cat2: مزهريات و أحواض نبات
Cat3: nan
Mater حجم المنتج   إصيص نباتات * عرض 49 * عمق 48 * ارتفاع 69 سم    ألياف زجاجية اللون   أبيض النوع   معاصر
next:  ♦ إصيص نباتات * عرض 49 * عمق 48 * ارتفاع 69 سم
Detail:  وصف المنتج  إصيص نباتات    إصيص نباتات 1 قطعة
Color:  حجم المنتج   إصيص نباتات * عرض 49 * عمق 48 * ارتفاع 69 سم مواد التصنيع   ألياف زجاجية    أبيض النوع   معاصر
['   إصيص نباتات ', ' عرض 49 ', ' عمق 48 ', ' ارتفاع 69 سم مواد التصنيع   ألياف زجاجية اللون   أبيض النوع   معاصر']
width 49
height 69
length 48
Style: حجم المنتج :  إصيص نباتات * عرض 49 * عمق 48 * ارتفاع 69 سم مواد التصنيع :  ألياف زجاجية اللون :  أبيض   معاصر
Code:  ce18d1d07b138f93805957ee70f15be2
*************************************
Count: 44
URL: https://midasfurniture.com/ksa_ar/covite-planter-small.html
It is ok
Cat1: إكسسوارات
Cat2: مزهريات

...

*************************************
Count: 673
URL: https://midasfurniture.com/ksa_ar/drana-artificial-tree.html
It is ok
*************************************
Count: 674
URL: https://midasfurniture.com/ksa_ar/reuben-artificial-tree.html
It is ok
*************************************
Count: 675
URL: https://midasfurniture.com/ksa_ar/lyoyd-artificial-tree.html
It is ok
*************************************
Count: 676
URL: https://midasfurniture.com/ksa_ar/griffiths-artificial-tree.html
It is ok
*************************************
Count: 677
URL: https://midasfurniture.com/ksa_ar/hopkins-artificial-tree.html
It is ok
*************************************
Count: 678
URL: https://midasfurniture.com/ksa_ar/terry-artificial-tree.html
It is ok
*************************************
Count: 679
URL: https://midasfurniture.com/ksa_ar/haynes-artificial-tree.html
It is ok
*************************************
Count: 680
URL: https://midasfurniture.com/ksa_ar/barks-artificial-tree.html
It is ok


*************************************
Count: 741
URL: https://midasfurniture.com/ksa_ar/vitix-table-lamp.html
It is ok
*************************************
Count: 742
URL: https://midasfurniture.com/ksa_ar/revex-table-lamp.html
It is ok
*************************************
Count: 743
URL: https://midasfurniture.com/ksa_ar/tragri-table-lamp.html
It is ok
*************************************
Count: 744
URL: https://midasfurniture.com/ksa_ar/crov-table-lamp.html
It is ok
*************************************
Count: 745
URL: https://midasfurniture.com/ksa_ar/ovex-table-lamp.html
It is ok
*************************************
Count: 746
URL: https://midasfurniture.com/ksa_ar/rondas-table-lamp.html
It is ok
*************************************
Count: 747
URL: https://midasfurniture.com/ksa_ar/verox-table-lamp.html
It is ok
*************************************
Count: 748
URL: https://midasfurniture.com/ksa_ar/milosh-table-lamp.html
It is ok
*************************************
Count: 74

*************************************
Count: 811
URL: https://midasfurniture.com/ksa_ar/lavaca-table-lamp.html
It is ok
*************************************
Count: 812
URL: https://midasfurniture.com/ksa_ar/hellmagic-table-lamp.html
It is ok
*************************************
Count: 813
URL: https://midasfurniture.com/ksa_ar/rapido-table-lamp.html
It is ok
*************************************
Count: 814
URL: https://midasfurniture.com/ksa_ar/harley-table-lamp.html
It is ok
*************************************
Count: 815
URL: https://midasfurniture.com/ksa_ar/humphrey-table-lamp.html
It is ok
*************************************
Count: 816
URL: https://midasfurniture.com/ksa_ar/barnes-table-lamp.html
It is ok
*************************************
Count: 817
URL: https://midasfurniture.com/ksa_ar/aren-table-lamp.html
It is ok
*************************************
Count: 818
URL: https://midasfurniture.com/ksa_ar/crawford-table-lamp.html
It is ok
***********************************

*************************************
Count: 880
URL: https://midasfurniture.com/ksa_ar/vitalco-hanging-lamp-small.html
It is ok
*************************************
Count: 881
URL: #
*************************************
Count: 882
URL: https://midasfurniture.com/ksa_ar/placido-hanging-lamp-large.html
It is ok
*************************************
Count: 883
URL: https://midasfurniture.com/ksa_ar/strollo-hanging-lamp-large.html
It is ok
*************************************
Count: 884
URL: https://midasfurniture.com/ksa_ar/brando-hanging-lamp-medium.html
It is ok
*************************************
Count: 885
URL: https://midasfurniture.com/ksa_ar/vitalco-hanging-lamp-medium.html
It is ok
*************************************
Count: 886
URL: https://midasfurniture.com/ksa_ar/bruland-hanging-lamp.html
It is ok
*************************************
Count: 887
URL: https://midasfurniture.com/ksa_ar/keron-hanging-lamp.html
It is ok
*************************************
Count: 888
URL: 

*************************************
Count: 952
URL: https://midasfurniture.com/ksa_ar/moopids-photo-frame.html
It is ok
*************************************
Count: 953
URL: https://midasfurniture.com/ksa_ar/wintil-photo-frame-small.html
It is ok
*************************************
Count: 954
URL: https://midasfurniture.com/ksa_ar/wintil-photo-frame-large.html
It is ok
*************************************
Count: 955
URL: https://midasfurniture.com/ksa_ar/alberic-photo-frame-small.html
It is ok
*************************************
Count: 956
URL: https://midasfurniture.com/ksa_ar/alberic-photo-frame-large.html
It is ok
*************************************
Count: 957
URL: https://midasfurniture.com/ksa_ar/kistop-photo-frame-small.html
It is ok
*************************************
Count: 958
URL: https://midasfurniture.com/ksa_ar/kistop-photo-frame-large.html
It is ok
*************************************
Count: 959
URL: https://midasfurniture.com/ksa_ar/duck-feet-photo-frame-8097

# Fin de fonction 

In [110]:
price = driver.find_elements_by_class_name('price')[0].get_attribute('innerHTML').replace('SAR', '').replace('\n', '').replace(' ', '')

special_price = driver.find_elements_by_class_name('price')[1].get_attribute('innerHTML').replace('SAR', '').replace('\n', '').replace(' ', '')

special_price, price

('65.00', '65.00')

In [232]:
url = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/djangoSripte/media/Midas/midas_update_products_url_25_09_2021v2.xlsx')

In [233]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/Midas_update_02_10_2021finalv12.xlsx')

In [234]:
list_url = []
for index, row in url.iterrows():
    list_url.append([ row['url'], row['cat1'], row['cat2'], row['cat3']])

In [254]:
for tt in list_url:
    df.loc[df['Link_url'] == tt[0], 'categories3'] = tt[3]

In [260]:
df.loc[df['categories3']=='']

,sku number only,sku,store_view_code,attribute_set_code,product_websites,name,estimated_delivery_enable,estimated_delivery_text,url_key,description,...,swatch_image,thumbnail_image,additional_images,product_online,qty,out_of_stock_qty,allow_backorders,is_in_stock,style,supplier
25,126348,DAS-126348,NaN,Default,base,كنبة زاوية دالهارت-,Static Text,هذا المنتج سيكون متوفر للتوصيل بعد 36 ايام-,كنبة زاوية دالهارت--DAS-126348,أريكة استرخاء يد يمين 1 قطعة كنبة ثلاثة مق...,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,2,0,-5,0,0,إنتقالى,DAS
33,122431,DAS-122431,NaN,Default,base,كنبة زاوية كيما,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,كنبة زاوية كيما-DAS-122431,كنبة ثلاثة مقاعد يد يمين 1 قطعة كنبة ثلاثة...,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
50,114390,DAS-114390,NaN,Default,base,كنبة زاوية بليميش,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,كنبة زاوية بليميش-DAS-114390,كنبة مقعدين يد يسار 1 قطعة كنبة مقعدين يد ...,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
51,114247,DAS-114247,NaN,Default,base,كنبة زاوية بليميش,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,كنبة زاوية بليميش-DAS-114247,كنبة مقعدين يد يسار 1 قطعة كنبة مقعدين يد ...,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
90,133376,DAS-133376,NaN,Default,base,طقم كنب ماكليري,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,طقم كنب ماكليري-DAS-133376,كنبة ثلاثة مقاعد بأيدي 1 قطعة كنبة مقعدين ...,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,إنتقالى,DAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5655,109365,DAS-109365,NaN,Default,base,وسادة ديكور إكسبو,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,وسادة ديكور إكسبو-DAS-109365,وسادة ديكور مربعة 1 قطعة,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
5657,109360,DAS-109360,NaN,Default,base,وسادة ديكور ميلوس,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,وسادة ديكور ميلوس-DAS-109360,وسادة ديكور مستطيلة 1 قطعة,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
5664,109328,DAS-109328,NaN,Default,base,وسادة ديكور فاكي,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,وسادة ديكور فاكي-DAS-109328,وسادة ديكور مستطيلة 1 قطعة,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
5692,123338,DAS-123338,NaN,Default,base,شال كور,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,شال كور-DAS-123338,شال 1 قطعة,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,إنتقالى,DAS


In [83]:
price = driver.find_elements_by_class_name('price')[0].get_attribute('innerHTML').replace('SAR', '').replace('\n', '').replace(' ', '')
special_price = driver.find_elements_by_class_name('price')[1].get_attribute('innerHTML').replace('SAR', '').replace('\n', '').replace(' ', '')
print(price, special_price)

265.00 185.50


# Traitement

In [35]:
df.drop_duplicates(subset=['sku'], inplace=True)

<ipython-input-35-e578a62fd7d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['sku'], inplace=True)


# delete all images with name logo 

In [346]:
df.loc[df['base_image'].str.contains('logo')] 

,sku,name,url_key,short_description,description,set_include,free_colors,raw_materials,ts_dimensions_width,ts_dimensions_height,...,attribute_set_code,product_type,store_view_code,supplier,style,First,Last,estimated_delivery_text,estimated_delivery_enable,sku number only


In [313]:
df = pd.read_csv('clean_data.csv')

In [79]:
df = None

In [80]:
df1 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-1.xlsx')
df9 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-2.xlsx')
df2 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-2-1.xlsx')
df3 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-3.xlsx')
df4 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-4.xlsx')
df8 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-4-1.xlsx')
df5 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-5.xlsx')
df6 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-6.xlsx')
df7 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-6-1.xlsx')
df10 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_update_14-07-22-7.xlsx')


In [82]:
df = pd.concat([df1, df2], ignore_index=True)

In [83]:
df = pd.concat([df, df3], ignore_index=True)

In [84]:
df = pd.concat([df, df4], ignore_index=True)

In [85]:
df = pd.concat([df, df5], ignore_index=True)

In [86]:
df = pd.concat([df, df6], ignore_index=True)

In [87]:
df = pd.concat([df, df7], ignore_index=True)

In [88]:
df = pd.concat([df, df8], ignore_index=True)

In [89]:
df = pd.concat([df, df9], ignore_index=True)

In [90]:
df = pd.concat([df, df10], ignore_index=True)

In [91]:
df = pd.concat([df, df11], ignore_index=True)

NameError: name 'df11' is not defined

In [93]:
df.shape

(6123, 54)

In [124]:
df.loc[df['price'].isnull()]

,sku,name,url_key,short_description,description,set_include,free_colors,raw_materials,products_size,ts_dimensions_width,...,manufacturer,news_from_date,news_to_date,product_websites,attribute_set_code,product_type,store_view_code,supplier,style,estimated_delivery_text


In [95]:
df.shape

(5833, 54)

In [94]:
df.drop_duplicates(subset=['sku'], inplace=True)

In [128]:
df.to_excel('clean_data.xlsx')

# Clean Fonctions

In [96]:
def clean_punct(name):
    for c in string.punctuation:
        df[name] = df[name].str.replace(c, '-', regex=False)
def toto_clean(name):
    df[name] = df[name].str.replace('•', '-')
    df[name] = df[name].str.replace(':', '-')
    df[name] = df[name].str.replace('/', '-')
    df[name] = df[name].str.replace('"', '-')
    df[name] = df[name].str.replace('%', '-')
    df[name] = df[name].str.replace('&', '-')
    df[name] = df[name].str.replace('?', '-', regex=False)
    df[name] = df[name].str.replace('(', '-', regex=False)
    df[name] = df[name].str.replace(')', '-', regex=False)
    df[name] = df[name].str.replace('{', '-', regex=False)
    df[name] = df[name].str.replace('}', '-', regex=False)
    df[name] = df[name].str.replace("'", '-', regex=False)
    df[name] = df[name].str.replace('*', 'X', regex=False)
    df[name] = df[name].str.replace('#', '-')
    df[name] = df[name].str.replace('”', '-')
    df[name] = df[name].str.replace('!', '-')
    df[name] = df[name].str.replace('♦', '-')
    df[name] = df[name].str.replace('♦', '-')
    df[name] = df[name].str.replace('=', '-')
    df[name] = df[name].str.replace('+', '-', regex=False)
    df[name] = df[name].str.replace('،', '-', regex=False)
    
def ampty_value(name):
    df.loc[df[name].isnull(), name] = '__EMPTY__VALUE__'
    
    
def price_num(name):
    try:
        df[name] = df[name].str.replace('-', '')
        df[name] = df[name].str.replace(',', '')
    except:
        pass
    df.loc[df[name] == '__EMPTY__VALUE__', name] = 0
    
    df[name] = pd.to_numeric(df[name])
    df.loc[df[name] == 0, name] = '__EMPTY__VALUE__'

In [99]:
toto_clean('name')
toto_clean('url_key')
clean_punct('name')
clean_punct('description')
toto_clean('description')
#toto_clean('short_description')
#clean_punct('short_description')
toto_clean('free_colors')
clean_punct('free_colors')
toto_clean('products_size')

,sku,name,url_key,short_description,description,set_include,free_colors,raw_materials,products_size,ts_dimensions_width,...,manufacturer,news_from_date,news_to_date,product_websites,attribute_set_code,product_type,store_view_code,supplier,style,estimated_delivery_text


In [100]:
df[['color','sec_color']] = df['free_colors'].str.split("-",expand=True,n = 1)

In [101]:
df.loc[df['sec_color']== 'شفاف', 'raw_materials'] = 'شفاف'
df.loc[df['color']== 'مراية', 'raw_materials'] = 'مراية'
df.loc[df['sec_color']== 'مراية', 'raw_materials'] = 'مراية'
df.loc[df['color']== 'شفاف', 'raw_materials'] = 'شفاف'
df.loc[df['color']== 'لون زجاجى', 'raw_materials'] = 'لون زجاجى'
df.loc[df['sec_color']== 'لون زجاجى', 'raw_materials'] = 'لون زجاجى'
df.loc[df['color']== ' زجاجى', 'raw_materials'] = ' زجاجى'
df.loc[df['sec_color']== ' زجاجى', 'raw_materials'] = ' زجاجى'


In [102]:
df.loc[df['color']== 'شفاف', 'color'] = '__EMPTY__VALUE__'
df.loc[df['color']== 'مراية', 'color'] = '__EMPTY__VALUE__'
df.loc[df['color']== 'لون زجاجى', 'color'] = '__EMPTY__VALUE__'
df.loc[df['color']== 'زجاجى', 'color'] = '__EMPTY__VALUE__'
df.loc[df['sec_color']== 'شفاف', 'sec_color'] = '__EMPTY__VALUE__'
df.loc[df['sec_color']== 'مراية', 'sec_color'] = '__EMPTY__VALUE__'
df.loc[df['sec_color']== 'لون زجاجى', 'sec_color'] = '__EMPTY__VALUE__'
df.loc[df['sec_color']== 'زجاجى', 'sec_color'] = '__EMPTY__VALUE__'

In [186]:
df.loc[df['color'].str.len() >= 30, 'color' ] = '__EMPTY__VALUE__'
df.loc[(df['free_colors'].str.contains('نظرة عامة')) | (df['free_colors'].isnull()) , 'color'] = '__EMPTY__VALUE__'
df.loc[df['color'] == '', 'color'] = '__EMPTY__VALUE__'
df.loc[df['color'].isnull(), 'color'] = '__EMPTY__VALUE__'
df.loc[df['sec_color'].isnull(), 'sec_color'] = '__EMPTY__VALUE__'
df.loc[df['sec_color'] == '', 'sec_color'] = '__EMPTY__VALUE__'
clean_punct('url_key')
df['description'] = df['description'].str.replace('♦', '', regex=False)
df['short_description'] = df['description']

In [139]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/clean_data.xlsx')

In [140]:
df.shape

(5912, 42)

# Add update files

In [105]:
df['price'] = df['price'].str.replace('KWD', '')
#df['special_price'] = df['special_price'].str.replace('KWD', '').str.replace('&nbsp;', '').str.replace("٬", '')

In [103]:
df['price'] = pd.to_numeric(df['price'])
#df['special_price'] = pd.to_numeric(df['special_price'])

In [42]:
df.drop(df[df['price'].isnull()].index, inplace=True)
df.drop(df[df['price']==0].index, inplace=True)

In [104]:
df.loc[df['price'].isnull()]

,sku,name,url_key,short_description,description,set_include,free_colors,raw_materials,products_size,ts_dimensions_width,...,weight,category1,category2,category3,raw_material,free_color1,style1,no_of_peices,color,sec_color


In [124]:
df.to_excel('toto_no_add_price.xlsx')

In [105]:
df['cost'] = df['price'] * 0.7
df['price1'] = df['price'] * 1.15
#df['special_price1'] = df['special_price'] * 1.15

In [106]:
df.shape

(5833, 58)

In [108]:
#df.loc[df['special_price1'] == df['price1'], 'special_price1'] = '__EMPTY__VALUE__'
df.loc[df['free_colors'].isnull(), 'free_colors'] = '__EMPTY__VALUE__'
df.loc[df['free_colors'] == ' لا', 'free_colors'] = '__EMPTY__VALUE__'
df['free_colors'] = df['free_colors'].str.replace(',', '-', regex=False)
df.loc[df['free_colors'].str.len() >= 30, 'free_colors' ] = '__EMPTY__VALUE__'
clean_punct('free_colors')
df.loc[df['Weight'] == 0, 'Weight'] = '__EMPTY__VALUE__'
toto_clean('raw_materials')
df.loc[(df['raw_materials'] == '') | (df['raw_materials'].isnull()), 'raw_materials'] = '__EMPTY__VALUE__'
df.loc[(df['ts_dimensions_width'] == '') | (df['ts_dimensions_width'].isnull()), 'ts_dimensions_width'] = '__EMPTY__VALUE__'
df.loc[(df['ts_dimensions_length'] == '') | (df['ts_dimensions_length'].isnull()), 'ts_dimensions_length'] = '__EMPTY__VALUE__'
df.loc[(df['ts_dimensions_height'] == '') | (df['ts_dimensions_height'].isnull()), 'ts_dimensions_height'] = '__EMPTY__VALUE__'
df.loc[(df['style'] == '') | (df['style'].isnull()), 'style'] = '__EMPTY__VALUE__'
#toto_clean('short_description')
df['categories'] = ''

In [39]:
df.to_excel('toto56.xlsx')

# ADD categories files

In [109]:
cats = pd.read_excel("/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/Categories/Midas category(1).xlsx")

In [110]:
list_cats = []
for index, row in cats.iterrows():
    list_cats.append([row['Name of category '], row['Num of category '], row['description']]) 

In [111]:
for tt in list_cats:
    df.loc[df['categories2'] == tt[0], 'categories'] =  tt[1]

In [112]:
for tt in list_cats:
    df.loc[df['categories3'] == tt[0], 'categories'] = tt[1]

In [113]:
for tt in list_cats:
    df.loc[(df['categories'] == tt[1]) & (df['description'].isnull()), 'description'] =  tt[2]

In [114]:
df.loc[df['categories'].isnull()]

,sku,name,url_key,short_description,description,set_include,free_colors,raw_materials,products_size,ts_dimensions_width,...,category3,raw_material,free_color1,style1,no_of_peices,color,sec_color,cost,price1,categories


In [115]:
for tt in list_cats:
    df.loc[(df['categories'] == tt[1]) & (df['description'].str.len() < 10), 'description'] =  tt[2]

In [116]:
df.loc[df['description'].str.len() < 10 ]

,sku,name,url_key,short_description,description,set_include,free_colors,raw_materials,products_size,ts_dimensions_width,...,category3,raw_material,free_color1,style1,no_of_peices,color,sec_color,cost,price1,categories


In [117]:
df['short_description'] = df['description']

In [153]:
df.to_excel("toto4.xlsx")

In [118]:
df.loc[df['free_colors'].isnull(), 'free_colors'] = '__EMPTY__VALUE__'

In [119]:
df['free_colors'] = df['free_colors'].str.replace('،', '-')

In [120]:
df['free_colors'] = df['free_colors'].str.replace(' لا', '__EMPTY__VALUE__')

In [121]:
df['raw_materials'] = df['raw_materials'].str.replace('/', '-')

In [122]:
df['free_colors'] = df['free_colors'].str.replace('/', '-', regex=False) 

# Split name 

In [123]:
df['sku'] = df['sku'].str.replace('المنتج', '')

In [124]:
df['url_key'] = df['name']+ '-' + df['sku']

In [125]:
df.to_excel('test_midas_clean_name.xlsx')

In [126]:
df.loc[~df['estimated_delivery_text'].isnull(), 'is_in_stock'] = 0

In [127]:
df.shape

(5833, 59)

In [128]:
df.drop(df[df['name'].isnull()].index, inplace=True)

In [144]:
df.loc[df['estimated_delivery_text'].isnull(), 'estimated_delivery_text'] =  'الوقت المقدر لشحن هذا المنتج أسبوعين'

In [145]:
df.loc[df['estimated_delivery_text'].isnull(), 'estimated_delivery_text']

Series([], Name: estimated_delivery_text, dtype: object)

In [149]:
df['estimated_delivery_enable'] = 'Static Text'

In [131]:
df.loc[df['is_in_stock'] == 'متوفر', 'is_in_stock'] = 1

In [132]:
df.loc[df['is_in_stock'] == 'غير متوفر', 'is_in_stock'] = 0

In [133]:
df['allow_backorders'] = df['is_in_stock']
df['product_online'] = df['is_in_stock']
df['qty'] = df['is_in_stock']  

In [134]:
df['product_online'] = df['is_in_stock']
df['qty'] = df['is_in_stock']  

In [135]:
df.loc[df['is_in_stock'] == 0, 'product_online'] = 2

In [136]:
df.loc[df['product_online'] == 2, 'out_of_stock_qty'] = 0
df.loc[df['product_online'] == 2, 'qty'] = 0
df.loc[df['product_online'] == 2, 'allow_backorders'] = 0
df.loc[df['product_online'] == 2, 'out_of_stock_qty'] = 0

In [137]:
# Check if special price is correct
for index, row in df.iterrows():
    if row['special_price'] != '__EMPTY__VALUE__':
        if row['special_price'] < row['price'] * 0.5 :
            print('ALERT----in', index, ': ', row['price'],'-' , row['special_price'], '-', row['Link_url'])

In [138]:
df.to_excel('toto3.xlsx')

In [139]:
df['name'] = df['name'].str.replace('\n', '')

In [140]:
df['sku number only'] = df['sku'].str.replace('DAS-', '')

In [141]:
df['product_type'] = 'simple'

In [150]:
df = df[['sku number only', 'sku', 'store_view_code', 'attribute_set_code', 
         'product_type',  'product_websites',
         'name',  'estimated_delivery_enable', 'estimated_delivery_text',  
         'url_key',  'description', 'short_description', 'products_size',
 'Link_url', 'categories1', 'categories2', 'categories3', 'categories',
         'set_include', 'color', 'sec_color',  'raw_materials','ts_dimensions_width', 
         'ts_dimensions_height', 'ts_dimensions_length',
             'raw_material',
            'free_color1',
            'style1',
            'no_of_peices',
    'cost',    
         'price1','price', 'special_price' , 'visibility', 'tax_class_name', 'manufacturer',
         'news_from_date', 'news_to_date', 'base_image', 'small_image', 'swatch_image' 
    , 'thumbnail_image', 'additional_images', 'product_online', 'qty', 'out_of_stock_qty', 'allow_backorders'
    , 'is_in_stock', 'style', 'supplier'
        ]]

In [142]:
df['special_price'] = '__EMPTY__VALUE__'

In [151]:
df.to_excel('Midas_update_16-07-2022v2.xlsx')

In [164]:
df.shape

(838, 46)

In [363]:
df['special_price'] = '__EMPTY__VALUE__'

In [114]:
df.loc[df['price'] == df['special_price'], 'special_price'] = 0

In [30]:
df.loc[df['style']=='Concept♦ معاصر']

,sku,name,url_key,short_description,description,free_colors,raw_materials,ts_dimensions_width,ts_dimensions_height,ts_dimensions_length,...,attribute_set_code,product_type,store_view_code,supplier,style,First,Last,estimated_delivery_text,estimated_delivery_enable,sku number only
997,DAS-114296,طاولة جانبية هولر,طاولة جانبية هولر-DAS-114296,طاولة جانبيه 1 قطعة,طاولة جانبيه 1 قطعة,Product ذهبى/ أبيض,إستانلس ستيل / رخام أبيض,46,53,46,...,Default,simple,,DAS,Concept♦ معاصر,طاولة جانبية هولر,None,الوقت المقدر لشحن هذا المنتج أسبوعين,Static Text,114296


In [99]:
6089.25 * 0.55


3349.0875

In [188]:
toto = df.loc[df['categories2']=='غرف نوم']

In [189]:
toto.to_excel('Midas_غرف نوم.xlsx')

In [116]:
df['categories'] = ''

In [204]:
df['sku number only'] = df['sku'].str.replace('DAS-', '')

In [242]:
df['sku number only'] 

0         9278
1        91285
2        88051
3         8180
4         2914
         ...  
6253    108989
6254    108988
6255    130773
6256    126790
6257    126788
Name: sku number only, Length: 5976, dtype: object

In [148]:
df['special_price'] = 

In [126]:
import datetime as dt
import xlsxwriter
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, colors
from openpyxl.styles.borders import Border, Side
from openpyxl.styles.alignment import Alignment
from openpyxl.styles.fills import PatternFill
import xlwt
from xlwt.Utils import cell_to_rowcol2
import datetime as dt
import excel
import csv

In [136]:
%%time
data = pd.read_excel("/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/Midas_update_31-05-2022v2.xlsx",sheet_name=None, engine="openpyxl")

CPU times: user 3.36 s, sys: 35.9 ms, total: 3.39 s
Wall time: 3.39 s


In [186]:
toto = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/Midas_update_25_09_2021finalv1.xlsx')

In [190]:
toto.loc[toto['sku'] == 'DAS-130672', 'Link_url'].to_list()

['https://midasfurniture.com/ksa_ar/bloss-office-visitor-chair.html']

In [221]:
df1 = pd.read_csv('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/midas_poppp.csv')

In [202]:
df.shape

(5632, 38)

In [204]:
skus = pd.read_excel('/home/wafistos/Downloads/ميداس.xlsx')

In [206]:
skus.shape

(126, 36)

In [207]:
list_skus = skus['sku'].to_list()

In [200]:
len(list_skus)

5599

In [213]:
df = None

In [222]:
df = df1.loc[df1['sku'].isin(list_skus)]

In [224]:
df.to_excel('tt.xlsx')

In [199]:
df.loc[df['sku'] == 'DAS-131069']

,sku,name,url_key,short_description,description,free_colors,raw_materials,ts_dimensions_width,ts_dimensions_height,ts_dimensions_length,...,allow_backorders,manufacturer,news_from_date,news_to_date,product_websites,attribute_set_code,product_type,store_view_code,supplier,style
3334,DAS-131069,حامل شموع كبير فينيارد,DAS-131069-حامل شموع كبير فينيارد,حامل شموع 1 قطعة,حامل شموع 1 قطعة,ذهبى، شفاف,معدن,11.0,41.0,10.0,...,1,مستوردة,09/27/2021,11/26/2021,base,Default,simple,NaN,DAS,معاصر


In [4]:
df= pd.read_excel('/home/wafistos/Downloads/Midas_test2.xlsx')

In [5]:
df['additional_images'] = df['additional_images'].str.replace(' ', '')

In [6]:
df.to_excel('Midas_test3.xlsx')

In [438]:
toto = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/djangoSripte/media/Tamkeen/TamkeenUpdate_all1.xlsx')

In [440]:
tata = toto.loc[toto['description'].isnull()]

In [ ]:
tata.to_excel

In [448]:
df = pd.DataFrame(data)

In [449]:
df.to_excel('dta.xlsx')

In [458]:
data[0]['sku']

'GSTN12H'

In [459]:
for tt in data:
 toto.loc[toto['sku'] == tt['sku'], 'description'] = tt['description']

In [460]:
toto.to_excel('testtoto.xlsx')

In [444]:
list_url = []
for index, row in tata.iterrows():
    list_url.append([ row['sku'], row['link_url']])

In [445]:
list_url 

[['GSTN12H', 'https://tamkeenstores.com.sa/product/gstn12h/'],
 ['ZCB19CAXFINNB', 'https://tamkeenstores.com.sa/product/zcb19caxfinnb/'],
 ['WWA20G8NHR', 'https://tamkeenstores.com.sa/product/wwa20g8nhr/'],
 ['GS1855C', 'https://tamkeenstores.com.sa/product/gs1855c/'],
 ['GSA50C', 'https://tamkeenstores.com.sa/product/gsa50c/'],
 ['VH-K110WBB', 'https://tamkeenstores.com.sa/product/vh-k110wbb/'],
 ['4KWFC120MAW', 'https://tamkeenstores.com.sa/product/4kwfc120maw/'],
 ['LX7021/2', 'https://tamkeenstores.com.sa/product/lx7021-2/'],
 ['LX7011/2', 'https://tamkeenstores.com.sa/product/lx7011-2/'],
 ['SE867GIFS', 'https://tamkeenstores.com.sa/product/se867gifs/'],
 ['JCD-882', 'https://tamkeenstores.com.sa/product/jcd-882/'],
 ['HD4646/70', 'https://tamkeenstores.com.sa/product/hd4646-70/'],
 ['HD9303/03', 'https://tamkeenstores.com.sa/product/hd9303-03/'],
 ['FKRCHTKUN91V', 'https://tamkeenstores.com.sa/product/fkrchtkun91v/'],
 ['HD2581/01', 'https://tamkeenstores.com.sa/product/hd2581-01

# Check price

In [2]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/Midas_update_25_10_2021finalv1.xlsx')

In [3]:
def check_price(categories):
    df1 = df.loc[df['categories'] == categories]
    df1['cost'] = df1['cost'].astype(float)
    toto = df1['cost'].min()
    return toto

In [6]:
pipo = check_price('43790,24,113')
pipo

<ipython-input-3-f4e51f231962>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['cost'] = df1['cost'].astype(float)


2500.0

In [9]:
cats = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/categories/Midas category(1).xlsx')

In [10]:
cats.columns

Index(['Name of category ', 'Num of category ', 'min', 'description',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')

In [21]:
cats['min'] = [check_price(x) for x in cats['Num of category ']]

<ipython-input-3-f4e51f231962>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['cost'] = df1['cost'].astype(float)


In [22]:
cats['min']

0      3150.0
1      4620.0
2      2500.0
3      1375.0
4       515.0
        ...  
121      18.0
122      11.0
123     310.0
124     450.0
125      28.0
Name: min, Length: 126, dtype: float64

In [23]:
cats.to_excel('Midas_categories.xlsx')

In [89]:
df

,sku number only,sku,store_view_code,attribute_set_code,product_type,product_websites,name,estimated_delivery_enable,estimated_delivery_text,url_key,...,swatch_image,thumbnail_image,additional_images,product_online,qty,out_of_stock_qty,allow_backorders,is_in_stock,style,supplier
0,9278,DAS-9278,,Default,simple,base,كنبة زاوية أكرينجتون,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,كنبة زاوية أكرينجتون-DAS-9278,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,إنتقالى,DAS
1,88051,DAS-88051,,Default,simple,base,طقم زاوية 2 قطعة - كرسي راحة هزاز تامبو,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,طقم زاوية 2 قطعة - كرسي راحة هزاز تامبو-DAS-88051,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,إنتقالى,DAS
2,8180,DAS-8180,,Default,simple,base,طقم كنب زاوية تراكلي,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,طقم كنب زاوية تراكلي-DAS-8180,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,إنتقالى,DAS
3,2353,DAS-2353,,Default,simple,base,طقم كنب زاوية لوكسورا,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,طقم كنب زاوية لوكسورا-DAS-2353,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,إنتقالى,DAS
4,2347,DAS-2347,,Default,simple,base,طقم كنب زاوية إينولا,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,طقم كنب زاوية إينولا-DAS-2347,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,إنتقالى,DAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6531,109513,DAS-109513,,Default,simple,base,خزنة ACS-4 اكزيكتيف,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,خزنة ACS-4 اكزيكتيف-DAS-109513,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,__EMPTY__VALUE__,DAS
6532,109512,DAS-109512,,Default,simple,base,خزنة CS-3 اكزيكتيف,Static Text,هذا المنتج سيكون متوفر للتوصيل بعد 89 ايام,خزنة CS-3 اكزيكتيف-DAS-109512,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,2,0,-5,0,0,__EMPTY__VALUE__,DAS
6728,130773,DAS-130773,,Default,simple,base,طقم مائدة 75 قطعة لينشو,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,طقم مائدة 75 قطعة لينشو-DAS-130773,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
6729,126790,DAS-126790,,Default,simple,base,طقم مائدة 75 قطعة روباني,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,طقم مائدة 75 قطعة روباني-DAS-126790,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS


In [103]:
toto = df.loc[(df['cost']<=200) | (df['categories3'] =='وسائد')]

In [104]:
toto.to_excel('Midas_filter_06_12.xlsx')

In [113]:
titi  = pd.read_excel('/home/wafistos/Downloads/ميداس 1-1-2022.xlsx')

In [139]:
df = pd.read_excel('/home/wafistos/Downloads/ميداس 1-1-2022(1).xlsx')

In [114]:
skus = titi['sku'].to_list()

In [115]:
len(skus)

57

In [133]:
df.loc[df['sku'].isin(skus)]

,sku number only,sku,store_view_code,attribute_set_code,product_type,product_websites,name,estimated_delivery_enable,estimated_delivery_text,url_key,...,swatch_image,thumbnail_image,additional_images,product_online,qty,out_of_stock_qty,allow_backorders,is_in_stock,style,supplier
51,113227,DAS-113227,NaN,Default,simple,base,كنبة زاوية جيستر,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,كنبة زاوية جيستر-DAS-113227,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
118,123129,DAS-123129,NaN,Default,simple,base,طقم كنب ديانثا,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,طقم كنب ديانثا-DAS-123129,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,إنتقالى,DAS
137,117088,DAS-117088,NaN,Default,simple,base,كينويك,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,كينويك-DAS-117088,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,__EMPTY__VALUE__,DAS
271,108538,DAS-108538,NaN,Default,simple,base,كنبة سرير 3 مقعد مع مرتبة دايا,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,كنبة سرير 3 مقعد مع مرتبة دايا-DAS-108538,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,__EMPTY__VALUE__,DAS
300,13192,DAS-13192,NaN,Default,simple,base,زوسين,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,زوسين-DAS-13192,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,__EMPTY__VALUE__,DAS
362,120895,DAS-120895,NaN,Default,simple,base,كرسى إستقبال كيندرا,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,كرسى إستقبال كيندرا-DAS-120895,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
524,123211,DAS-123211,NaN,Default,simple,base,اوتومان فلوريا,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,اوتومان فلوريا-DAS-123211,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
618,16680,DAS-16680,NaN,Default,simple,base,باري ليني,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,باري ليني-DAS-16680,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,__EMPTY__VALUE__,DAS
656,112673,DAS-112673,NaN,Default,simple,base,فيرسيلي,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,فيرسيلي-DAS-112673,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS
783,114281,DAS-114281,NaN,Default,simple,base,طاولة وسط رودى,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,طاولة وسط رودى-DAS-114281,...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,https://cdn.midasfurniture.com/media/catalog/p...,1,0,-5,1,1,معاصر,DAS


In [134]:
df1 = df.loc[df['sku'].isin(skus)]

In [135]:
df1['free_colors']

51                   رمادى
118             بيج- رمادى
137       --EMPTY--VALUE--
271       --EMPTY--VALUE--
300       --EMPTY--VALUE--
362        أزرق فاتح- ذهبى
524           ألوان متنوعة
618       --EMPTY--VALUE--
656             فضي - أسود
783                    فضى
791                    بيج
843                  مراية
1087             بنى- أسود
1205            بنى- أسود 
1253                  أسود
1270            ذهبى- أبيض
1324                  بيج 
1325                  بيج 
1326              بنى فاتح
1327             بنى داكن 
1328               رمادي  
1329               رمادي  
1368            أبيض- ذهبى
1408      --EMPTY--VALUE--
1414         أبيض - رمادي 
1416                مراية 
1417      --EMPTY--VALUE--
1419           ذهبى- أبيض 
1480              بنى فاتح
1617                   بنى
1623           كريمى- ذهبى
1670             بيج- ذهبى
1765                  ذهبى
1953              بيج- فضى
1954           كريمي- ذهبى
1955           كريمي- ذهبى
1956        بنى فاتح- ذهبى
1

In [122]:
df1.columns

Index(['Unnamed: 0', 'sku number only', 'sku', 'store_view_code',
       'attribute_set_code', 'product_type', 'product_websites', 'name',
       'estimated_delivery_enable', 'estimated_delivery_text', 'url_key',
       'description', 'short_description', 'Link_url', 'categories1',
       'categories2', 'categories3', 'categories', 'free_colors',
       'raw_materials', 'ts_dimensions_width', 'ts_dimensions_height',
       'ts_dimensions_length', 'cost', 'price', 'special_price', 'visibility',
       'tax_class_name', 'manufacturer', 'news_from_date', 'news_to_date',
       'base_image', 'small_image', 'swatch_image', 'thumbnail_image',
       'additional_images ', 'product_online', 'qty', 'out_of_stock_qty',
       'allow_backorders', 'is_in_stock', 'style', 'supplier'],
      dtype='object')

In [143]:
df[['color','sec_color']] = df['free_colors'].str.split("-",expand=True,n = 1)

In [159]:
df.to_excel('midas1111.xlsx')